# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\130.83.248.207\ValidationTests\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
wmg.Sessions

#0: OscillatingDroplet3D	OD3D_J1024k3_refinedTest_amr0_case2_Oh0.1_StartUp*	03/25/2022 08:31:29	fb0bffd1...
#1: OscillatingDroplet3D	OD3D_J3456k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly*	03/16/2022 13:38:16	a4a8b9b2...
#2: OscillatingDroplet3D	OD3D_J432k3_pressureOutletBC_amr1_case1_Oh0.1_AnalytInitPhaseAonly*	03/16/2022 13:38:06	0120c987...
#3: OscillatingDroplet3D	OD3D_J3456k3_wallBC_amr0_case1_Oh0.1_AnalytInitPhaseAonly*	03/10/2022 12:42:43	9ced7020...
#4: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case1_Oh0.1*	03/10/2022 10:06:47	85cf2911...
#5: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case1_Oh0.1_AnalytInitPhaseAonly_Newton*	03/09/2022 11:27:59	99091aea...
#6: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case1_Oh0.1_AnalytInitPhaseAonly*	03/09/2022 11:27:43	a5ab589b...


In [ ]:
var studySess = wmg.Sessions.Skip(6);
studySess

#0: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp	03/17/2022 11:36:55	cd996c49...
#1: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp	03/17/2022 11:37:29	8362baba...
#2: OscillatingDroplet3D	OD3D_J432k3_pressureOutletBC_amr1_case2_Oh0.1_StartUp	03/17/2022 15:44:05	8d327352...
#3: OscillatingDroplet3D	OD3D_J432k3_pressureOutletBC_amr1_case3_Oh0.1_StartUp	03/17/2022 15:44:36	9ddfe34a...
#4: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_switchedComponents_AnalytInitPhaseAonly	03/21/2022 15:21:29	6ed4b40d...
#5: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case4_Oh0.1_StartUp_switchedComponents_AnalytInitPhaseAonly	03/21/2022 15:22:32	d9b53054...
#6: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case5_Oh0.56_StartUp_switchedComponents_AnalytInitPhaseAonly	03/21/2022 15:23:37	3eab8c83...
#7: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case4_Oh0.1_StartUp_initCase1_AnalytInitPhaseAonly	03/21/2022 12:02:41	b1f333c4...
#8: OscillatingDropl

In [ ]:
//databases.Pick(0).Sessions

In [ ]:
//var targetDB = databases.Pick(0);
//targetDB
//databases.Pick(1).Sessions.MoveAll(targetDB);
//wmg.Sessions[0].Move(targetDB);
//wmg.Sessions.Where(s => s.Name.Contains("J432k3_arm0_case2_Oh0.1_AnalytInit"))
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()

In [ ]:
int sNum = 42;
wmg.Sessions[sNum]

OscillatingDroplet3D	J432k3_amr0_case5_Oh0.56_AnalytInit	02/21/2022 13:25:25	faa12478...

In [ ]:
wmg.Sessions[sNum].GetSessionDirectory()

\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\23a6e85a-e66a-45d8-8b8b-919a6cc3de91

In [ ]:
wmg.Sessions[sNum].DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Feb21_132417

In [ ]:
wmg.Sessions[sNum].Timesteps.Count

801

In [ ]:
wmg.Sessions[sNum].Timesteps.Export().WithSupersampling(3).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__J432k3_amr0_case5_Oh0.56_AnalytInit__faa12478-cb4d-49fe-9213-3c76bb20e7f0


In [ ]:
using System.IO;

# Study Overview

In [ ]:
//string[] modes = new string[]{ "mode2", "mode3", "mode4"};
//string[] aPis = new string[] { "aP0", "aP1", "aP2" };
//string[] amrS = new string[] { "amr0" };
string[] caseS = new string[] { "case1", "case2", "case3", "case4", "case5" };
string[] BCs = new string[] { "wallBC" }; //{ "wallBC", "pressureOutletBC" };
bool[] ShouldAnaInit = new bool[] { false, true };
string[] InitCases = new string[] {"_initCase0", "_initCase1", "_initCase2", "_switchedComponents", "_"};
bool[] useNewton = new bool[] { false };

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
//string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iBC = 0; iBC < BCs.Length; iBC++) {    
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iInit = 0; iInit < InitCases.Length; iInit++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
    //string _amr = amrS[iAMR];
    string _case = caseS[iRow];
    string _bc = BCs[iBC];
    string _init = InitCases[iInit];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                    //   && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains(_bc))
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains(_init))
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(_init == "_") {
        SIs = SIs.Where(sess => !sess.Name.Contains("_initCase")
                                && !sess.Name.Contains("_switchedComponents")
                                && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna]));
    }

    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        // var dummyPlot = new Plot2Ddata();
        // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        // PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        fmt.DashType = DashTypes.Dashed;
    } else {  
        fmt.LineColor = LineColors.Blue;
        fmt.DashType = DashTypes.Solid;
    }
    if (!ShouldAnaInit[iAna]) 
        fmt.LineColor = LineColors.Red;
    
    if (_init == "_initCase0")
        fmt.LineColor = LineColors.Green;
    if (_init == "_initCase1")
        fmt.LineColor = LineColors.Magenta;
    if (_init == "_initCase2")
        fmt.LineColor = LineColors.Orange;
    if (_init == "_switchedComponents")
        fmt.LineColor = LineColors.Grey;
 

    string name = "BoSSS-"+_bc;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-Ainit" + _init;
    if(useNewton[iNew])
        name = name + "-Newton";

    double[] L = tab["L"].ToArray(); 
    double[] Wx = tab["Wx"].ToArray(); 
    double[] Wy = tab["Wy"].ToArray();
    double[] arLW = new double[L.Length];
    for (int i = 0; i < L.Length; i++) {
        arLW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
    }
    
    List<double> arLW_lst = new List<double>();
    List<double> time_lst = new List<double>();
    double dataThreshold = 0.25;
    for(int i = 0; i < L.Length; i++) {
        if(arLW[i] > dataThreshold) {
            arLW_lst.Add(arLW[i]);
            time_lst.Add(time[i]);
        }
    }
    arLW = arLW_lst.ToArray();
    time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0)
       PlotTableDM[iRow,iCol].Ylabel = caseS[iRow];
}
}
}
}
}
}

OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp
OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_initCase0_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_initCase1_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_initCase2_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_switchedComponents_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp
OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp_AnalytInitPhaseAonly
The process cannot access the file '\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\49e4fabb-0859-4f4d-8662-67324f32c4d3\DropletMetrics.txt' because it is being used by another process.
Skipping OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp_AnalytInitPhaseAonly*	03/17/2022 11:33:25	49e4fabb...
OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp
OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr0_case4_Oh0.1

In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case1 <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case2 <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case3 <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case4 <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case5 <null> <null>

## Case 1

In [ ]:
//var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/Oh_0dot1_LWinTime.txt");
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/2_Oh_1_0dot4_AspRatioInTime.txt");
var basaran_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/basaranOscOh_0dot1.txt");
var becker_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/beckerEtAlOscOh_0dot1.txt");
var meradji_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/meradjiOscOh_0dot1.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[0,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Magenta;
//PlotTableDM[0,0].AddDataGroup("Basaran (1992)", basaran_dat.GetColumn(0), basaran_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Green;
//PlotTableDM[0,0].AddDataGroup("Becker et. al. (1994)", becker_dat.GetColumn(0), becker_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Orange;
//PlotTableDM[0,0].AddDataGroup("Meradji et. al. (2001)", meradji_dat.GetColumn(0), meradji_dat.GetColumn(1), fmt);

//var gp = dat_plot.ToGnuplot();
//gp.PlotSVG()

In [ ]:
PlotTableDM[0,0].XrangeMin = 0;
PlotTableDM[0,0].XrangeMax = 0.5;
var gp = PlotTableDM[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 2 
 
 
 
 
 2.2 
 
 
 
 
 2.4 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 BoSSS-wallBC 
 
 
 
 
 BoSSS-wallBC 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M288.8,57.1 L342.2,57.1 M71.9,261.6 L78.7,261.7 L85.5,261.0 L92.2,260.8 L99.0,261.2 L105.8,261.7
 L112.6,262.3 L119.4,263.1 L126.2,264.0 L132.9,264.9 L139.7,265.9 L146.5,266.9 L153.3,268.0 L160.1,269.1
 L166.8,270.2 L173.6,271.3 L180.4,272.6 L187.2,274.0 L194.0,275.4 L200.8,276.7 L207.5,278.1 L214.3,279.5
 L221.1,280.9 L227.9,282.3 L234.7,283.7 L241.5,285.2 L248.2,286.6 L255.0,288.1 L261.8,289.5 L268.6,291.0
 L275.4,292.5 L282.1,294.0 L288.9,295.5 L295.7,297.0 L302.5,298.6 L309.3,300.1 L316.1,301.7 L322.8,303.2
 L329.6,304.8 L336.4,306.4 L343.2,308.0 L350.0,309.5 L356.7,311.1 L363.5,312.7 L370.3,314.4 L377.1,316.0
 L383.9,317.6 L390.7,319.2 L397.4,320.8 L404.2,322.5 L411.0,324.1 L417.8,325.8 L424.6,327.4 L431.3,329.1
 L438.1,330.7 L444.9,332.4 L451.7,334.1 L458.5,335.7 L465.3,337.4 L472.0,339.1 L478.8,340.8 L485.6,342.5
 L492.4,344.2 L499.2,345.9 L505.9,347.7 L512.7,349.4 L519.5,351.1 L526.3,352.9 L533.1,354.7 L539.9,356.4
 L546.6,358.2 L553.4,360.0 L560.2,361.8 L567.0,363.7 L573.8,365.5 L580.6,367.3 L587.3,369.2 L594.1,371.1
 L600.9,373.0 L607.7,374.9 L614.5,376.8 L621.2,378.7 L628.0,380.7 L634.8,382.6 L641.6,384.6 L648.4,386.6
 L655.2,388.6 L661.9,390.7 L668.7,392.7 L675.5,394.8 L682.3,396.9 L689.1,398.9 L695.8,401.0 L702.6,403.1
 L709.4,405.3 L716.2,407.4 L723.0,409.6 L729.8,411.8 L736.5,414.0 L743.3,416.2 L750.1,418.4 '/> 
 
 BoSSS-wallBC-AinitinitCase0 
 
 
 BoSSS-wallBC-Ainit i nitCase0 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M288.8,81.1 L342.2,81.1 M71.9,261.6 L78.7,240.6 L85.5,245.0 L92.2,248.7 L99.0,252.0 L105.8,255.0
 L112.6,257.9 L119.4,260.7 L126.2,263.3 L132.9,265.8 L139.7,268.2 L146.5,270.4 L153.3,272.5 L160.1,273.7
 L166.8,275.8 L173.6,277.9 L180.4,279.8 L187.2,281.7 L194.0,283.4 L200.8,285.1 L207.5,286.8 L214.3,288.4
 L221.1,290.2 L227.9,291.8 L234.7,293.5 L241.5,295.0 L248.2,296.6 L255.0,298.1 L261.8,299.7 L268.6,301.2
 L275.4,303.1 L282.1,305.0 L288.9,306.5 L295.7,308.0 L302.5,309.5 L309.3,311.0 L316.1,312.4 L322.8,313.9
 L329.6,315.7 L336.4,317.1 L343.2,318.4 L350.0,319.8 L356.7,321.1 L363.5,322.4 L370.3,323.7 L377.1,325.0
 L383.9,326.3 L390.7,327.6 L397.4,328.9 L404.2,330.2 L411.0,331.5 L417.8,332.9 L424.6,334.2 L431.3,335.6
 L438.1,337.0 L444.9,338.8 L451.7,340.3 L458.5,341.8 L465.3,343.4 L472.0,345.0 L478.8,346.6 L485.6,348.3
 L492.4,350.0 L499.2,351.7 L505.9,353.5 L512.7,355.3 L519.5,357.2 L526.3,359.1 L533.1,361.1 L539.9,363.0
 L546.6,365.0 L553.4,367.0 L560.2,369.1 L567.0,371.1 L573.8,373.2 L580.6,375.3 L587.3,377.4 L594.1,379.5
 L600.9,381.7 L607.7,383.8 L614.5,385.9 L621.2,388.0 L628.0,390.1 L634.8,392.2 L641.6,394.3 L648.4,396.5
 L655.2,398.6 L661.9,400.7 L668.7,402.7 L675.5,404.8 L682.3,406.9 L689.1,409.0 L695.8,411.0 L702.6,412.9
 L709.4,414.9 L716.2,416.8 L723.0,418.7 L729.8,420.6 L736.5,422.6 L743.3,424.5 L750.1,426.4 '/> 
 
 BoSSS-wallBC-AinitinitCase1 
 
 
 BoSSS-wallBC-Ainit i nitCase1 
 
 
 
	<path stroke='rgb(255, 0, 255)' d='M288.8,105.1 L342.2,105.1 M71.9,261.6 L78.7,287.6 L92.2,265.3 L99.0,254.0 L105.8,243.1 L112.6,233.9
 L119.4,226.8 L146.5,181.2 L160.1,159.4 L166.8,146.6 L173.6,147.5 L180.4,149.8 L187.2,153.2 L194.0,157.2
 L200.8,164.2 L207.5,170.2 L214.3,181.4 L221.1,193.1 L268.6,217.3 L288.9,244.6 L302.5,248.6 L309.3,250.6
 L322.8,254.4 L329.6,256.3 L336.4,260.2 L343.2,262.0 L350.0,264.0 L

In [ ]:
foreach(var grp in PlotTableDM[0,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case1/StartUpStudy/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 2

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/3_Oh_1_0dot4_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[1,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[1,0].XrangeMin = 0;
PlotTableDM[1,0].XrangeMax = 0.5;
var gp = PlotTableDM[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 1.02 
 
 
 
 
 1.04 
 
 
 
 
 1.06 
 
 
 
 
 1.08 
 
 
 
 
 1.1 
 
 
 
 
 1.12 
 
 
 
 
 1.14 
 
 
 
 
 1.16 
 
 
 
 
 1.18 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case2 
 
 
 
 
 BoSSS-wallBC 
 
 
 
 
 BoSSS-wallBC 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M544.1,57.1 L597.5,57.1 M82.9,556.7 L85.6,552.9 L96.3,542.7 L99.0,540.1 L101.6,537.5 L104.3,536.4
 L107.0,533.3 L109.7,530.0 L112.4,527.9 L115.0,524.3 L117.7,520.7 L120.4,517.0 L123.1,513.2 L125.8,509.5
 L128.4,511.0 L131.1,505.5 L133.8,500.0 L136.5,495.2 L139.2,492.3 L141.8,487.6 L144.5,483.4 L147.2,479.3
 L149.9,475.3 L152.5,471.4 L155.2,467.6 L157.9,463.8 L160.6,460.0 L163.3,456.2 L165.9,452.4 L168.6,436.7
 L171.3,433.9 L179.3,410.2 L182.0,407.3 L184.7,410.5 L187.4,403.2 L200.8,409.7 L203.5,406.8 L214.2,388.9
 L216.9,386.9 L219.5,384.7 L222.2,382.5 L232.9,341.8 L251.7,345.3 L254.4,342.9 L257.1,340.4 L259.7,317.1
 L267.8,308.8 L291.9,270.8 L294.6,268.3 L297.2,266.8 L299.9,265.0 L302.6,265.2 L305.3,262.5 L308.0,260.2
 L310.6,258.2 L313.3,256.3 L316.0,254.5 L318.7,252.7 L321.4,252.1 L324.0,250.0 L326.7,248.1 L329.4,246.4
 L332.1,245.8 L334.8,230.4 L337.4,236.7 L340.1,236.2 L342.8,240.0 L345.5,236.7 L348.2,235.5 L350.8,228.3
 L353.5,228.3 L356.2,224.4 L358.9,224.8 L361.6,222.2 L364.2,221.5 L366.9,219.3 L372.3,217.4 L375.0,224.8
 L377.6,222.5 L380.3,220.4 L383.0,219.7 L385.7,217.9 L388.4,216.6 L391.0,214.5 L401.8,198.1 L404.4,197.2
 L407.1,196.3 L409.8,195.3 L412.5,194.3 L415.2,193.2 L431.2,185.8 L441.9,179.0 L444.6,175.8 L447.3,173.1
 L450.0,170.7 L452.7,168.6 L458.0,168.6 L460.7,166.2 L484.8,151.0 L487.5,146.2 L514.3,140.8 L517.0,139.2
 L519.7,129.2 L522.3,128.5 L525.0,127.9 L527.7,127.3 L530.4,126.9 L533.1,126.6 L541.1,126.2 L543.8,126.2
 L546.5,126.7 L549.1,126.7 L551.8,126.4 L554.5,126.9 L557.2,127.6 L559.8,128.3 L581.3,129.1 L589.3,133.8
 L594.7,136.8 L600.0,139.7 L605.4,142.5 L608.1,144.2 L618.8,153.3 L621.5,154.8 L624.2,156.3 L626.8,154.3
 L629.5,152.9 L632.2,153.5 L634.9,155.5 L637.6,156.9 L640.2,158.4 L642.9,164.5 L645.6,166.2 L648.3,167.9
 L651.0,169.7 L653.6,167.2 L656.3,171.0 L659.0,175.0 L661.7,176.8 L664.4,171.9 L667.0,180.5 L669.7,182.3
 L672.4,182.6 L691.1,191.9 L693.8,193.8 L734.0,410.4 '/> 
 
 WNLT 
 
 
 WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M544.1,81.1 L597.5,81.1 M80.2,564.0 L93.6,561.1 L107.0,552.7 L120.4,539.4 L133.8,521.8 L147.2,500.3
 L160.6,475.7 L174.0,448.5 L187.4,419.4 L200.8,389.0 L214.2,357.9 L227.6,326.5 L241.0,295.4 L254.4,265.2
 L267.8,236.1 L281.2,208.6 L294.6,182.9 L308.0,159.4 L321.4,138.3 L334.8,119.7 L348.2,103.6 L361.6,90.3
 L375.0,79.6 L388.4,71.5 L401.8,66.0 L415.2,63.0 L428.5,62.3 L441.9,63.9 L455.3,67.4 L468.7,72.8
 L482.1,79.9 L495.5,88.4 L508.9,98.2 L522.3,109.0 L535.7,120.7 L549.1,133.0 L562.5,145.9 L575.9,159.0
 L589.3,172.4 L602.7,185.7 L616.1,198.9 L629.5,211.8 L642.9,224.3 L656.3,236.4 L669.7,247.9 L683.1,258.8
 L696.5,269.0 L709.9,278.5 L723.3,287.2 L736.7,295.1 L750.1,302.2 '/>

In [ ]:
foreach(var grp in PlotTableDM[1,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case2/StartUpStudy/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 3

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/4_Oh_1_0dot4_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[2,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[2,0].XrangeMin = 0;
PlotTableDM[2,0].XrangeMax = 0.5;
var gp = PlotTableDM[2,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 1.25 
 
 
 
 
 1.3 
 
 
 
 
 1.35 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case3 
 
 
 
 
 BoSSS-wallBC 
 
 
 
 
 BoSSS-wallBC 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M477.5,57.1 L530.9,57.1 M80.2,201.9 L82.9,202.6 L85.6,202.6 L88.2,201.9 L90.9,200.8 L96.3,200.3
 L99.0,200.2 L101.6,200.0 L104.3,199.6 L107.0,199.3 L109.7,198.9 L112.4,198.6 L115.0,198.2 L117.7,197.7
 L120.4,197.2 L123.1,196.7 L125.8,196.2 L128.4,195.7 L131.1,195.2 L133.8,194.7 L136.5,194.1 L139.2,193.6
 L141.8,193.2 L144.5,192.7 L147.2,192.3 L149.9,191.8 L152.5,191.4 L155.2,190.9 L157.9,190.4 L160.6,190.0
 L163.3,189.6 L165.9,189.2 L168.6,188.8 L171.3,188.5 L174.0,188.1 L176.7,187.8 L179.3,187.4 L182.0,189.2
 L184.7,186.7 L187.4,186.4 L190.1,186.1 L192.7,185.8 L195.4,185.6 L198.1,185.4 L200.8,185.2 L203.5,186.7
 L206.1,185.8 L208.8,198.2 L211.5,201.0 L214.2,201.7 L216.9,202.2 L219.5,202.4 L222.2,202.5 L224.9,202.5
 L227.6,202.5 L230.3,202.5 L232.9,201.1 L235.6,201.1 L238.3,201.0 L241.0,199.0 L243.7,199.2 L246.3,198.8
 L249.0,198.6 L251.7,200.8 L254.4,202.1 L257.1,201.2 L259.7,201.3 L262.4,201.2 L265.1,197.7 L267.8,200.8
 L270.5,200.9 L273.1,201.2 L275.8,201.3 L278.5,201.2 L281.2,201.0 L283.8,200.9 L286.5,200.6 L289.2,198.4
 L291.9,198.8 L294.6,199.5 L297.2,199.7 L299.9,197.4 L302.6,197.9 L305.3,196.9 L308.0,196.7 L310.6,197.3
 L313.3,198.0 L316.0,198.2 L318.7,198.3 L321.4,200.3 L324.0,200.0 L326.7,199.9 L329.4,197.5 L332.1,197.4
 L334.8,197.3 L337.4,197.3 L340.1,197.3 L342.8,197.4 L345.5,197.5 L348.2,197.7 L350.8,198.0 L353.5,196.9
 L356.2,198.6 L358.9,198.2 L361.6,198.9 L364.2,202.3 L366.9,202.5 L369.6,201.9 L372.3,203.1 L375.0,203.6
 L377.6,201.9 L380.3,204.0 L383.0,205.9 L385.7,207.0 L388.4,207.9 L391.0,208.8 L393.7,209.8 L396.4,210.9
 L399.1,212.1 L401.8,213.2 L404.4,214.7 L407.1,216.2 L409.8,215.1 L412.5,215.0 L415.2,218.0 L417.8,220.7
 L420.5,223.0 L423.2,225.1 L425.9,227.2 L428.5,229.3 L431.2,231.6 L433.9,235.0 L436.6,237.6 L439.3,239.6
 L441.9,242.0 L444.6,244.6 L447.3,244.2 L450.0,248.6 L452.7,252.3 L455.3,255.8 L458.0,259.0 L460.7,262.4
 L463.4,265.7 L466.1,269.1 L468.7,272.5 L471.4,276.0 L474.1,279.6 L476.8,283.2 L479.5,286.8 L482.1,290.5
 L484.8,289.5 L487.5,285.7 L490.2,289.2 L492.9,292.7 L495.5,296.5 L498.2,300.5 L500.9,304.6 L503.6,308.8
 L506.3,313.1 L508.9,317.4 L511.6,321.7 L514.3,326.1 L517.0,330.5 L519.7,334.3 L522.3,338.6 L525.0,342.9
 L527.7,347.2 L530.4,351.6 L533.1,355.4 L535.7,359.9 L538.4,364.1 L541.1,368.6 L543.8,373.0 L546.5,376.8
 L549.1,381.2 L551.8,385.7 L554.5,389.9 L557.2,394.5 L559.8,398.3 L562.5,402.2 L565.2,405.2 L567.9,408.1
 L570.6,411.1 L573.2,414.1 L575.9,417.2 L578.6,420.1 L581.3,423.0 L584.0,426.3 L586.6,429.5 L589.3,432.8
 L592.0,435.0 L594.7,438.3 L597.4,441.2 L600.0,444.1 L602.7,447.3 L605.4,450.4 L608.1,453.5 L610.8,456.6
 L613.4,459.7 L616.1,462.7 L618.8,465.7 L621.5,468.6 L624.2,470.4 L626.8,472.1 L656.3,499.3 L659.0,500.9
 L661.7,502.1 L664.4,502.7 L667.0,503.6 L669.7,504.6 L672.4,505.5 L675.1,507.1 L677.8,507.1 L680.4,507.2
 L683.1,507.6 L685.8,509.0 L688.5,508.5 L691.1,508.1 L693.8,508.1 L696.5,509.1 L699.2,508.3 L701.9,507.5
 L704.5,507.1 L707.2,507.8 L709.9,506.6 L712.6,505.8 L715.3,504.9 L717.9,504.1 L720.6,504.4 L723.3,502.8
 L726.0,501.7 L728.7,500.2 L731.3,499.3 L734.0,495.9 '/> 
 
 BoSSS-wallBC-Ainit 
 
 
 BoSSS-wallBC-Ainit 
 
 
 
 
 
 WNLT 
 
 
 WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M477.5,105.1 L530.9,105.1 M80.2,206.5 L93.6,207.8 L107.

## Case 4

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/2_Oh_1_0dot2_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[3,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[3,0].XrangeMin = 0;
PlotTableDM[3,0].XrangeMax = 0.5;
var gp = PlotTableDM[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 1.25 
 
 
 
 
 1.3 
 
 
 
 
 1.35 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case4 
 
 
 
 
 BoSSS-wallBC 
 
 
 
 
 BoSSS-wallBC 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M288.8,57.1 L342.2,57.1 M80.2,131.0 L86.9,130.2 L93.6,133.1 L100.3,131.5 L107.0,130.8 L113.7,130.6
 L120.4,132.5 L127.1,131.2 L133.8,131.9 L140.5,132.7 L147.2,133.6 L153.9,134.6 L160.6,135.6 L167.3,136.7
 L174.0,137.9 L180.7,139.1 L187.4,140.5 L194.1,141.9 L200.8,143.5 L207.5,145.1 L214.2,146.8 L220.9,148.5
 L227.6,150.3 L234.3,152.1 L241.0,154.0 L247.7,156.0 L254.4,158.0 L261.1,160.1 L267.8,162.3 L274.5,164.5
 L281.2,166.8 L287.9,169.1 L294.6,171.5 L301.3,173.9 L308.0,176.4 L314.7,178.9 L321.4,181.5 L328.1,184.1
 L334.8,186.8 L341.5,189.5 L348.2,192.3 L354.9,195.1 L361.6,198.0 L368.3,200.9 L375.0,203.8 L381.7,206.9
 L388.4,209.9 L395.1,213.0 L401.8,216.2 L408.5,220.7 L415.2,223.9 L421.8,226.7 L428.5,229.7 L435.2,232.9
 L441.9,236.1 L448.6,239.5 L455.3,242.9 L462.0,246.4 L468.7,249.9 L475.4,253.5 L482.1,257.1 L488.8,260.8
 L495.5,264.5 L502.2,268.3 L508.9,272.1 L515.6,276.0 L522.3,279.9 L529.0,283.9 L535.7,287.9 L542.4,291.9
 L549.1,296.0 L555.8,300.1 L562.5,304.3 L569.2,308.5 L575.9,312.7 L582.6,317.0 L589.3,321.3 L596.0,325.7
 L602.7,330.1 L609.4,334.6 L616.1,339.0 L622.8,343.6 L629.5,348.1 L636.2,352.7 L642.9,357.4 L649.6,362.0
 L656.3,366.8 L663.0,371.5 L669.7,376.3 L676.4,381.1 L683.1,385.9 L689.8,390.8 L696.5,395.7 L703.2,400.6
 L709.9,405.5 L716.6,410.5 L723.3,415.5 L730.0,420.5 L736.7,425.6 L743.4,430.6 L750.1,435.7 '/> 
 
 BoSSS-wallBC-AinitinitCase0 
 
 
 BoSSS-wallBC-Ainit i nitCase0 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M288.8,81.1 L342.2,81.1 M80.2,131.0 L86.9,100.6 L93.6,111.0 L100.3,120.5 L107.0,128.2 L113.7,134.4
 L120.4,139.4 L127.1,143.6 L133.8,147.1 L140.5,150.1 L147.2,152.7 L153.9,154.9 L160.6,157.0 L167.3,158.9
 L174.0,160.7 L180.7,162.4 L187.4,164.0 L194.1,165.6 L200.8,167.2 L207.5,168.8 L214.2,170.4 L220.9,172.1
 L227.6,173.7 L234.3,175.3 L241.0,176.9 L247.7,178.6 L254.4,180.2 L261.1,181.8 L267.8,183.5 L274.5,185.1
 L281.2,186.7 L287.9,188.2 L294.6,189.8 L301.3,191.4 L308.0,192.9 L314.7,194.4 L321.4,196.0 L328.1,197.5
 L334.8,199.0 L341.5,200.5 L348.2,202.0 L354.9,203.5 L361.6,205.1 L368.3,206.7 L375.0,208.3 L381.7,209.9
 L388.4,211.6 L395.1,213.4 L401.8,213.4 L408.5,215.4 L415.2,219.2 L421.8,221.3 L428.5,223.3 L435.2,225.5
 L441.9,227.8 L448.6,230.2 L455.3,232.6 L462.0,235.2 L468.7,237.9 L475.4,240.6 L482.1,243.5 L488.8,246.4
 L495.5,249.5 L502.2,252.6 L508.9,255.9 L515.6,259.2 L522.3,262.6 L529.0,266.2 L535.7,269.8 L542.4,273.5
 L549.1,277.2 L555.8,281.1 L562.5,285.0 L569.2,289.0 L575.9,293.1 L582.6,297.1 L589.3,301.2 L596.0,305.3
 L602.7,309.5 L609.4,313.8 L616.1,318.1 L622.8,322.4 L629.5,326.8 L636.2,331.3 L642.9,335.8 L649.6,340.3
 L656.3,344.9 L663.0,349.5 L669.7,354.2 L676.4,358.9 L683.1,363.6 L689.8,368.3 L696.5,373.1 L703.2,377.9
 L709.9,382.7 L716.6,387.5 L723.3,392.4 L730.0,397.2 L736.7,402.1 L743.4,407.0 L750.1,411.9 '/> 
 
 BoSSS-wallBC-AinitinitCase1 
 
 
 BoSSS-wallBC-Ainit i nitCase1 
 
 
 
	<path stroke='rgb(255, 0, 255)' d='M288.8,105.1 L342.2,105.1 M80.2,131.0 L86.9,170.5 L93.6,159.1 L100.3,142.6 L107.0,130.0 L113.7,120.2
 L120.4,111.9 L127.1,106.2 L133.8,101.7 L140.5,98.4 L147.2,96.1 L153.9,94.7 L160.6,94.0 L167.3,93.1
 L174.0,92.9 L180.7,94.0 L187.4,95.4 L194.1,97.2 L200.8,99.2 L207.5,101.3 L214.2,103.7 L220.9,106.2
 L227.6,108.8 L234.3,111.6 L241.0,114.5 L247.7,117.6 L254.4,120.

In [ ]:
foreach(var grp in PlotTableDM[3,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case4/StartUpStudy/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 5

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/4_Oh_56_0dot05_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[4,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[4,0].XrangeMin = 0;
PlotTableDM[4,0].XrangeMax = 0.5;
var gp = PlotTableDM[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 1.03 
 
 
 
 
 1.04 
 
 
 
 
 1.05 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case5 
 
 
 
 
 BoSSS-wallBC 
 
 
 
 
 BoSSS-wallBC 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M288.8,57.1 L342.2,57.1 M80.2,180.9 L86.9,182.1 L93.6,184.8 L100.3,184.8 L107.0,184.8 L113.7,185.0
 L120.4,185.3 L127.1,185.8 L133.8,186.4 L140.5,187.3 L147.2,188.2 L153.9,189.3 L160.6,190.6 L167.3,192.0
 L174.0,193.5 L180.7,195.1 L187.4,196.8 L194.1,198.6 L200.8,200.4 L207.5,202.3 L214.2,204.3 L220.9,206.4
 L227.6,208.4 L234.3,231.6 L241.0,233.2 L247.7,234.9 L254.4,236.7 L261.1,238.5 L267.8,240.5 L274.5,242.5
 L281.2,244.6 L287.9,246.6 L294.6,248.8 L301.3,250.9 L308.0,253.1 L314.7,255.3 L321.4,257.4 L328.1,259.6
 L334.8,261.8 L341.5,264.0 L348.2,266.2 L354.9,268.3 L361.6,270.5 L368.3,272.6 L375.0,274.7 L381.7,276.8
 L388.4,278.8 L395.1,280.9 L401.8,282.9 L408.5,284.9 L415.2,286.8 L421.8,288.7 L428.5,290.6 L435.2,292.4
 L441.9,294.3 L448.6,296.1 L455.3,297.8 L462.0,299.5 L468.7,301.2 L475.4,302.9 L482.1,304.5 L488.8,306.1
 L495.5,307.7 L502.2,309.2 L508.9,310.7 L515.6,312.2 L522.3,313.7 L529.0,315.1 L535.7,316.4 L542.4,317.8
 L549.1,319.1 L555.8,320.4 L562.5,321.7 L569.2,322.9 L575.9,324.1 L582.6,325.3 L589.3,326.4 L596.0,327.5
 L602.7,328.6 L609.4,329.7 L616.1,330.7 L622.8,331.7 L629.5,332.7 L636.2,333.7 L642.9,334.6 L649.6,335.6
 L656.3,336.5 L663.0,337.3 L669.7,338.2 L676.4,339.0 L683.1,339.8 L689.8,340.5 L696.5,341.1 L703.2,341.7
 L709.9,342.3 L716.6,342.9 L723.3,343.4 L730.0,344.0 L736.7,344.5 L743.4,345.1 L750.1,345.4 '/> 
 
 BoSSS-wallBC-AinitinitCase0 
 
 
 BoSSS-wallBC-Ainit i nitCase0 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M288.8,81.1 L342.2,81.1 M80.2,180.9 L86.9,202.2 L93.6,206.0 L100.3,206.7 L107.0,209.5 L113.7,208.6
 L120.4,205.8 L127.1,202.9 L133.8,201.4 L140.5,199.8 L147.2,195.0 L153.9,196.8 L160.6,195.4 L167.3,194.1
 L174.0,189.6 L180.7,188.7 L187.4,187.9 L194.1,187.2 L200.8,186.6 L207.5,186.0 L214.2,185.6 L220.9,185.2
 L227.6,184.9 L234.3,184.7 L241.0,184.5 L247.7,184.4 L254.4,184.4 L261.1,184.4 L267.8,184.5 L274.5,184.6
 L281.2,184.8 L287.9,185.0 L294.6,185.2 L301.3,185.5 L308.0,185.9 L314.7,186.2 L321.4,186.6 L328.1,187.1
 L334.8,187.5 L341.5,188.0 L348.2,188.5 L354.9,189.1 L361.6,189.6 L368.3,190.2 L375.0,190.8 L381.7,191.4
 L388.4,192.0 L395.1,192.6 L401.8,193.2 L408.5,193.9 L415.2,194.5 L421.8,195.1 L428.5,195.5 L435.2,196.1
 L441.9,196.8 L448.6,197.5 L455.3,198.3 L462.0,199.1 L468.7,199.9 L475.4,200.6 L482.1,201.4 L488.8,202.2
 L495.5,203.0 L502.2,203.8 L508.9,214.8 L515.6,225.7 L522.3,226.3 L529.0,226.9 L535.7,227.5 L542.4,228.2
 L549.1,228.9 L555.8,229.8 L562.5,230.9 L569.2,231.9 L575.9,232.9 L582.6,234.0 L589.3,235.0 L596.0,235.9
 L602.7,236.9 L609.4,237.8 L616.1,238.8 L622.8,239.7 L629.5,240.5 L636.2,241.4 L642.9,242.3 L649.6,243.1
 L656.3,243.9 L663.0,244.8 L669.7,245.6 L676.4,246.4 L683.1,247.2 L689.8,247.9 L696.5,248.7 L703.2,249.5
 L709.9,250.2 L716.6,251.0 L723.3,251.7 L730.0,252.5 L736.7,253.2 L743.4,253.9 L750.1,254.6 '/> 
 
 BoSSS-wallBC-AinitinitCase1 
 
 
 BoSSS-wallBC-Ainit i nitCase1 
 
 
 
	<path stroke='rgb(255, 0, 255)' d='M288.8,105.1 L342.2,105.1 M80.2,180.9 L86.9,158.8 L93.6,178.9 L100.3,173.3 L107.0,168.9 L113.7,167.5
 L120.4,168.1 L127.1,169.9 L133.8,172.4 L140.5,175.4 L147.2,178.8 L153.9,182.5 L160.6,186.3 L167.3,190.4
 L174.0,194.6 L180.7,198.9 L187.4,203.3 L194.1,207.8 L200.8,212.4 L207.5,217.0 L214.2,221.7 L220.9,228.1
 L227.6,237.2 L234.3,242.0 L

In [ ]:
foreach(var grp in PlotTableDM[4,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case5/StartUpStudy/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

# Plotting of Spherical Harmonics

In [ ]:
bool skipEqualModes = false;
bool skipUnequalModes = true;
bool skipNonPoly = true; // false: plots all Legendgre functions, i.e. indices (*, *)
//                          true:  plots only Legendre polynomials, i.e. inices (*, 0)

In [ ]:
List<Plot2Ddata>[] PlotTableSH = new List<Plot2Ddata>[caseS.Length];
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
//for(int iAMR = 0; iAMR < amrS.Length; iAMR++) {
for(int iBC = 0; iBC < BCs.Length; iBC++) {    
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
    //string _amr = amrS[iAMR];
    string _case = caseS[iRow];
    string _bc = BCs[iBC];

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      //&& sess.Name.Contains(_amr)
                                      && sess.Name.Contains(_bc)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    SIs = SIs.Where(sess => !sess.Name.Contains("_initCase")
                                      && !sess.Name.Contains("_switchedComponents")
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna]));

    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
        //PlotTableSH[iRow] = new List<Plot2Ddata>();
        if(PlotTableSH[iRow] == null) 
            PlotTableSH[iRow] = new List<Plot2Ddata>();
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        //var dummyPlot = new Plot2Ddata();
        //dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        if(PlotTableSH[iRow] == null) 
            PlotTableSH[iRow] = new List<Plot2Ddata>();
        continue;
    }


    var time = tab["time"];
    int index = 0;
    foreach(var column in tab) {
       
        var plot = new Plot2Ddata();

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines; 
        
        if(column.Key == "time")
            continue;
        if(skipNonPoly) {
            if(!column.Key.Contains(", 0)")) {
                continue;
            }
        }
        // if(skipEqualModes) {
        //     if(column.Key.Contains("(0, 0)") || column.Key.Contains("(2, 0)") 
        //     || column.Key.Contains("(4, 0)") || column.Key.Contains("(6, 0)") 
        //     || column.Key.Contains("(8, 0)") || column.Key.Contains("(10, 0)") 
        //     || column.Key.Contains("(12, 0)")) {
        //         continue;
        //     }
        // }
        if(iRow != 1) { // corresponds to case 2 with m=3
            if(column.Key.Contains("(1, 0)") || column.Key.Contains("(3, 0)") 
            || column.Key.Contains("(5, 0)") || column.Key.Contains("(7, 0)") 
            || column.Key.Contains("(9, 0)") || column.Key.Contains("(11, 0)")) {
                continue;
            }
        }
            
        if (useNewton[iNew]) {
            fmt.DashType = DashTypes.Dashed;
            fmt.LineColor = LineColors.Red;
        } else {  
            fmt.DashType = DashTypes.Solid;
            fmt.LineColor = LineColors.Blue;
        }
        if (!ShouldAnaInit[iAna]) 
        fmt.LineColor = LineColors.Red;

        string name = column.Key;
        if(name.Contains(", 0)"))
            fmt.LineWidth = 3;
        // if(iAMR > 0)
        //     name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";

        plot.ShowLegend = true;

        plot.Ylabel = column.Key;
        plot.XrangeMin = 0;
        plot.XrangeMax = 0.5;


        if(name.Contains("(0, 0)")) {
            List<double> procVal = new List<double>();
            foreach(double val in column.Value) {
                procVal.Add(val-1);
            }
            plot.AddDataGroup(name, time, procVal, fmt);
        } else    
            plot.AddDataGroup(name, time, column.Value, fmt);


        if (PlotTableSH[iRow].Count > index) {   
            PlotTableSH[iRow].ElementAt(index).AddDataGroup(plot.dataGroups[0]);
        } else {
            PlotTableSH[iRow].Add(plot);
        }

        index++;
   }
   
    
//    if(iAMR == 0) {
//        PlotTableSH[iRow,iCol] = plot;
//    } else {
//        PlotTableSH[iRow,iCol] = PlotTableSH[iRow,iCol].Merge(plot);
//    }
   
}
}
}
}

OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp
OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_AnalytInitPhaseAonly
OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp
The process cannot access the file '\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\cd996c49-b7ba-4ea3-95e8-eda1ba0476f4\SphericalHarmonics.txt' because it is being used by another process.
Skipping OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp*	03/17/2022 11:36:55	cd996c49...
OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp_AnalytInitPhaseAonly
The process cannot access the file '\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\49e4fabb-0859-4f4d-8662-67324f32c4d3\SphericalHarmonics.txt' because it is being used by another process.
Skipping OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp_AnalytInitPhaseAonly*	03/17/2022 11:33:25	49e4fabb...
OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp
The process cannot access the file '\\130.83.248.207\ValidationTests\OscillatingDroplet3D\ses

In [ ]:
PlotTableSH[1]

(empty)

### comparison data WNLT

In [ ]:
string[] caseNames = new string[] {"2_Oh1_0dot4", "3_Oh1_0dot4", "4_Oh1_0dot4", "2_Oh1_0dot2"};

In [ ]:
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int i = 0; i < PlotTableSH[iRow].Count; i++) {
    MultidimensionalArray dat;
    if(iRow == 1) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/{caseNames[iRow]}_plotRM{i}.txt");
        } catch {
            continue;
        }
    } else {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/{caseNames[iRow]}_plotRM{2*i}.txt");
        } catch {
            continue;
        }
    }

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    fmt.DashType = DashTypes.Dashed;
    fmt.LineColor = LineColors.Black;
    if(iRow == 1) {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    } else {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({2*i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    }
}
}

In [ ]:
for(int i = 0; i < PlotTableSH[4].Count; i++) {
    MultidimensionalArray dat;
    try {
        dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/4_Oh56_plotDataMode{2*i}.txt");
    } catch {
        continue;
    }

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    fmt.DashType = DashTypes.Dashed;
    fmt.LineColor = LineColors.Black;
    PlotTableSH[4].ElementAt(i).AddDataGroup($"({2*i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
}

### Case 1

In [ ]:
var gp = PlotTableSH[0].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.04 
 
 
 
 
 -0.035 
 
 
 
 
 -0.03 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M1526.1,60.1 L1579.5,60.1 M106.4,218.2 L122.7,218.1 L138.9,217.9 L155.2,217.8 L171.4,217.5 L187.7,217.2
 L203.9,216.9 L220.2,216.5 L236.5,216.1 L252.7,215.6 L269.0,215.0 L285.2,214.4 L301.5,213.8 L317.7,213.1
 L334.0,212.3 L350.3,211.5 L366.5,210.7 L382.8,209.8 L399.0,208.9 L415.3,207.9 L431.5,206.9 L447.8,205.9
 L464.1,204.8 L480.3,203.6 L496.6,202.5 L512.8,201.3 L529.1,200.0 L545.3,198.8 L561.6,197.5 L577.9,196.1
 L594.1,194.8 L610.4,193.3 L626.6,191.9 L642.9,190.5 L659.1,189.0 L675.4,187.5 L691.7,185.9 L707.9,184.3
 L724.2,182.7 L740.4,181.1 L756.7,179.5 L772.9,177.8 L789.2,176.1 L805.5,174.4 L821.7,172.7 L838.0,171.0
 L854.2,169.2 L870.5,167.4 L886.7,165.6 L903.0,163.8 L919.3,162.0 L935.5,160.2 L951.8,158.3 L968.0,156.5
 L984.3,154.6 L1000.5,152.7 L1016.8,150.8 L1033.0,148.9 L1049.3,147.0 L1065.6,145.1 L1081.8,143.2 L1098.1,141.2
 L1114.3,139.3 L1130.6,137.4 L1146.8,135.4 L1163.1,133.4 L1179.4,131.5 L1195.6,129.5 L1211.9,127.6 L1228.1,125.6
 L1244.4,123.6 L1260.6,121.7 L1276.9,119.7 L1293.2,117.7 L1309.4,115.8 L1325.7,113.8 L1341.9,111.9 L1358.2,109.9
 L1374.4,108.0 L1390.7,106.0 L1407.0,104.1 L1423.2,102.1 L1439.5,100.2 L1455.7,98.3 L1472.0,96.3 L1488.2,94.4
 L1504.5,92.5 L1520.8,90.6 L1537.0,88.7 L1553.3,86.9 L1569.5,85.0 L1585.8,83.1 L1602.0,81.3 L1618.3,79.4
 L1634.6,77.6 L1650.8,75.8 L1667.1,74.0 L1683.3,72.2 L1699.6,70.5 L1715.8,68.7 L1732.1,67.0 '/> 
 
 (0, 0)-Ainit 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,84.1 L1579.5,84.1 M106.4,218.2 L122.7,219.5 L138.9,220.6 L155.2,221.9 L171.4,222.7 L187.7,224.3
 L203.9,225.2 L220.2,225.9 L236.5,226.7 L252.7,227.4 L269.0,228.2 L285.2,228.4 L301.5,229.2 L317.7,229.8
 L334.0,230.5 L350.3,231.2 L366.5,231.7 L382.8,232.1 L399.0,232.5 L415.3,232.9 L431.5,233.1 L447.8,233.3
 L464.1,233.4 L480.3,233.4 L496.6,233.6 L512.8,233.6 L529.1,233.5 L545.3,233.2 L561.6,233.0 L577.9,233.7
 L594.1,235.4 L610.4,235.1 L626.6,234.8 L642.9,234.8 L659.1,235.3 L675.4,235.0 L691.7,236.1 L707.9,237.6
 L724.2,236.5 L740.4,234.3 L756.7,234.7 L772.9,234.6 L789.2,233.7 L805.5,233.3 L821.7,232.8 L838.0,232.2
 L854.2,232.6 L870.5,232.5 L886.7,232.5 L903.0,232.3 L919.3,232.2 L935.5,231.7 L951.8,231.6 L968.0,231.4
 L984.3,231.3 L1000.5,231.0 L1016.8,230.8 L1033.0,230.4 L1049.3,230.1 L1065.6,229.8 L1081.8,229.6 L1098.1,229.4
 L1114.3,229.3 L1130.6,229.0 L1146.8,228.9 L1163.1,228.7 L1179.4,228.6 L1195.6,228.4 L1211.9,228.2 L1228.1,228.0
 L1244.4,227.8 L1260.6,227.5 L1276.9,227.2 L1293.2,226.9 L1309.4,226.6 L1325.7,226.2 L1341.9,225.9 L1358.2,225.5
 L1374.4,225.1 L1390.7,224.7 L1407.0,224.3 L1423.2,223.8 L1439.5,223.3 L1455.7,222.8 L1472.0,222.3 L1488.2,221.8
 L1504.5,221.2 L1520.8,220.7 L1537.0,220.1 L1553.3,219.4 L1569.5,218.8 L1585.8,218.1 L1602.0,217.4 L1618.3,216.7
 L1634.6,216.0 L1650.8,215.2 L1667.1,214.4 L1683.3,213.6 L1699.6,212.8 L1715.8,211.9 L1732.1,211.1 '/> 
 
 (0, 0)-WNLT 
 
 
 (0, 0)-WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' stroke-dasharray='7.5,12.0' d='M1526.1,108.1 L1579.5,108.1 M106.4,218.1 L138.9,218.0 L171.4,217.8 L203.9,217.4 L236.5,216.8 L269.0,216.1
 L301.5,215.3 L334.0,214.3 L366.5,213.1 L399.0,211.7 L431.5,210.2 L464.1,208.6 L496.6,206.7 L529.1,204.8
 L561.6,202.6 L594.1,200.3 L626.6,197.8 L659.1,195.1 L691.7,192.3 L724.2,189.4 L756.7,186.2 L789.2,182.9
 

In [ ]:
foreach(var lst in PlotTableSH[0]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case1/StartUpStudy/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

### Case 2

In [ ]:
var gp = PlotTableSH[1].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Gnuplot Error:            line 0: undefined variable: Infinity




<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0

### Case 3

In [ ]:
var gp = PlotTableSH[2].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.03 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 
 WNLT 
 
 
 WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' stroke-dasharray='7.5,12.0' d='M1526.1,84.1 L1579.5,84.1 M106.4,186.0 L110.5,185.5 L114.5,184.0 L118.6,181.6 L122.7,178.2 L126.7,174.0
 L130.8,169.0 L134.8,163.3 L138.9,157.0 L143.0,150.2 L147.0,143.0 L151.1,135.6 L155.2,128.0 L159.2,120.4
 L163.3,113.0 L167.4,105.9 L171.4,99.2 L175.5,92.9 L179.6,87.4 L183.6,82.5 L187.7,78.4 L191.7,75.0
 L195.8,72.5 L199.9,70.9 L203.9,70.0 L208.0,69.8 L212.1,70.3 L216.1,71.5 L220.2,73.1 L224.3,75.1
 L228.3,77.4 L232.4,79.9 L236.5,82.5 L240.5,85.1 L244.6,87.5 L248.6,89.8 L252.7,91.8 L256.8,93.5
 L260.8,94.8 L264.9,95.8 L269.0,96.3 L273.0,96.5 L277.1,96.3 L281.2,95.7 L285.2,94.8 L289.3,93.6
 L293.4,92.2 L297.4,90.5 L301.5,88.8 L305.5,86.9 L309.6,85.0 L313.7,83.1 L317.7,81.3 L321.8,79.5
 L325.9,77.9 L329.9,76.4 L334.0,75.1 L338.1,73.9 L342.1,73.0 L346.2,72.2 L350.3,71.6 L354.3,71.2
 L358.4,70.9 L362.4,70.8 L366.5,70.9 L370.6,71.0 L374.6,71.3 L378.7,71.6 L382.8,72.1 L386.8,72.5
 L390.9,73.0 L395.0,73.5 L399.0,74.0 L403.1,74.4 L407.2,74.9 L411.2,75.3 L415.3,75.6 L419.3,75.9
 L423.4,76.1 L427.5,76.2 L431.5,76.3 L435.6,76.3 L439.7,76.3 L443.7,76.1 L447.8,76.0 L451.9,75.7
 L455.9,75.4 L460.0,75.1 L464.1,74.8 L468.1,74.4 L472.2,74.0 L476.2,73.7 L480.3,73.3 L484.4,72.9
 L488.4,72.6 L492.5,72.2 L496.6,71.9 L500.6,71.7 L504.7,71.4 L508.8,71.2 L512.8,71.1 L516.9,70.9
 L521.0,70.9 L525.0,70.8 L529.1,70.8 L533.1,70.8 L537.2,70.9 L541.3,70.9 L545.3,71.0 L549.4,71.1
 L553.5,71.2 L557.5,71.3 L561.6,71.4 L565.7,71.5 L569.7,71.7 L573.8,71.7 L577.9,71.8 L581.9,71.9
 L586.0,72.0 L590.0,72.0 L594.1,72.0 L598.2,72.0 L602.2,72.0 L606.3,72.0 L610.4,71.9 L614.4,71.9
 L618.5,71.8 L622.6,71.7 L626.6,71.7 L630.7,71.6 L634.8,71.5 L638.8,71.4 L642.9,71.3 L646.9,71.2
 L651.0,71.2 L655.1,71.1 L659.1,71.0 L663.2,71.0 L667.3,70.9 L671.3,70.9 L675.4,70.9 L679.5,70.8
 L683.5,70.8 L687.6,70.8 L691.7,70.8 L695.7,70.8 L699.8,70.8 L703.8,70.8 L707.9,70.9 L712.0,70.9
 L716.0,70.9 L720.1,70.9 L724.2,71.0 L728.2,71.0 L732.3,71.0 L736.4,71.0 L740.4,71.0 L744.5,71.1
 L748.6,71.1 L752.6,71.1 L756.7,71.1 L760.7,71.1 L764.8,71.1 L768.9,71.1 L772.9,71.1 L777.0,71.0
 L781.1,71.0 L785.1,71.0 L789.2,71.0 L793.3,71.0 L797.3,71.0 L801.4,70.9 L805.5,70.9 L809.5,70.9
 L813.6,70.9 L817.6,70.9 L821.7,70.9 L825.8,70.9 L829.8,70.8 L833.9,70.8 L838.0,70.8 L842.0,70.8
 L846.1,70.8 L850.2,70.8 L854.2,70.8 L858.3,70.8 L862.4,70.8 L866.4,70.8 L870.5,70.8 L874.5,70.8
 L878.6,70.8 L882.7,70.8 L886.7,70.8 L890.8,70.8 L894.9,70.9 L898.9,70.9 L903.0,70.9 L907.1,70.9
 L911.1,70.9 L915.2,70.9 L919.3,70.9 L923.3,70.9 L927.4,70.9 L931.4,70.9 L935.5,70.9 L939.6,70.9
 L943.6,70.9 L947.7,70.9 L951.8,70.9 L955.8,70.9 L959.9,70.8 L964.0,70.8 L968.0,70.8 L972.1,70.8
 L976.1,70.8 L980.2,70.8 L984.3,70.8 L988.3,70.8 L992.4,70.8 L996.5,70.8 L1000.5,70.8 L1004.6,70.8
 L1008.7,70.8 L1012.7,70.8 L1016.8,70.8 L1020.9,70.8 L1024.9,70.8 L1029.0,70.8 L1033.0,70.8 L1037.1,70.8
 L1041.2,70.8 L1045.2,70.8 L1049.3,70.8 L1053.4,70.8 L1057.4,70.8 L1061.5,70.8 L1065.6,70.8 L1069.6,70.8
 L1073.7,70.8 L1077.8,70.8 L1081.8,70.8 L1085.9,70.8 L1089.9,70.8 L1094.0,70.8 L1098.1,70.8 L1102.1,70.8
 L1106.2,70.8 L1110.3,70.8 L1114.3,70.8 L1118.4,70.8 L1122.5,70.8 L1126.5,70.8 L1130.6,70.8 L1134.7,70.8


### Case 4

In [ ]:
var gp = PlotTableSH[3].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.009 
 
 
 
 
 -0.008 
 
 
 
 
 -0.007 
 
 
 
 
 -0.006 
 
 
 
 
 -0.005 
 
 
 
 
 -0.004 
 
 
 
 
 -0.003 
 
 
 
 
 -0.002 
 
 
 
 
 -0.001 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M1526.1,60.1 L1579.5,60.1 M106.4,240.5 L122.7,239.9 L138.9,239.5 L155.2,239.1 L171.4,238.7 L187.7,238.3
 L203.9,237.9 L220.2,237.4 L236.5,236.8 L252.7,236.2 L269.0,235.5 L285.2,234.8 L301.5,234.0 L317.7,233.1
 L334.0,232.2 L350.3,231.2 L366.5,230.2 L382.8,229.0 L399.0,227.9 L415.3,226.7 L431.5,225.4 L447.8,224.0
 L464.1,222.7 L480.3,221.2 L496.6,219.7 L512.8,218.2 L529.1,216.6 L545.3,215.0 L561.6,213.3 L577.9,211.5
 L594.1,209.8 L610.4,207.9 L626.6,206.1 L642.9,204.2 L659.1,202.2 L675.4,200.2 L691.7,198.2 L707.9,196.1
 L724.2,194.0 L740.4,191.9 L756.7,189.7 L772.9,187.5 L789.2,185.3 L805.5,183.0 L821.7,180.7 L838.0,178.4
 L854.2,176.1 L870.5,173.7 L886.7,171.3 L903.0,170.5 L919.3,168.0 L935.5,165.0 L951.8,162.1 L968.0,159.5
 L984.3,156.9 L1000.5,154.4 L1016.8,151.8 L1033.0,149.3 L1049.3,146.7 L1065.6,144.2 L1081.8,141.7 L1098.1,139.1
 L1114.3,136.6 L1130.6,134.0 L1146.8,131.5 L1163.1,128.9 L1179.4,126.4 L1195.6,123.8 L1211.9,121.3 L1228.1,118.7
 L1244.4,116.2 L1260.6,113.7 L1276.9,111.2 L1293.2,108.7 L1309.4,106.2 L1325.7,103.7 L1341.9,101.2 L1358.2,98.7
 L1374.4,96.3 L1390.7,93.9 L1407.0,91.5 L1423.2,89.1 L1439.5,86.7 L1455.7,84.4 L1472.0,82.0 L1488.2,79.7
 L1504.5,77.5 L1520.8,75.2 L1537.0,73.0 L1553.3,70.8 L1569.5,68.6 L1585.8,66.5 L1602.0,64.4 L1618.3,62.3
 L1634.6,60.2 L1650.8,58.2 L1667.1,56.2 L1683.3,54.3 L1699.6,52.4 L1715.8,50.5 L1732.1,48.7 '/> 
 
 (0, 0)-Ainit 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,84.1 L1579.5,84.1 M106.4,240.5 L122.7,240.0 L138.9,239.6 L155.2,239.3 L171.4,239.1 L187.7,238.8
 L203.9,238.5 L220.2,238.1 L236.5,237.7 L252.7,237.2 L269.0,236.7 L285.2,236.1 L301.5,235.4 L317.7,234.7
 L334.0,233.9 L350.3,233.0 L366.5,232.1 L382.8,231.2 L399.0,230.1 L415.3,229.1 L431.5,227.9 L447.8,226.7
 L464.1,225.5 L480.3,224.2 L496.6,222.8 L512.8,221.4 L529.1,220.0 L545.3,218.5 L561.6,216.9 L577.9,215.3
 L594.1,213.7 L610.4,212.0 L626.6,210.3 L642.9,208.5 L659.1,206.7 L675.4,204.8 L691.7,202.9 L707.9,201.0
 L724.2,199.0 L740.4,197.0 L756.7,195.0 L772.9,192.9 L789.2,190.8 L805.5,188.6 L821.7,186.5 L838.0,184.3
 L854.2,182.0 L870.5,179.8 L886.7,177.5 L903.0,175.2 L919.3,172.9 L935.5,170.5 L951.8,168.2 L968.0,167.4
 L984.3,164.9 L1000.5,161.9 L1016.8,159.1 L1033.0,156.5 L1049.3,153.9 L1065.6,151.4 L1081.8,148.9 L1098.1,146.4
 L1114.3,143.9 L1130.6,141.4 L1146.8,138.9 L1163.1,136.4 L1179.4,133.9 L1195.6,131.4 L1211.9,128.8 L1228.1,126.3
 L1244.4,123.8 L1260.6,121.3 L1276.9,118.8 L1293.2,116.3 L1309.4,113.8 L1325.7,111.3 L1341.9,108.8 L1358.2,106.3
 L1374.4,103.9 L1390.7,101.4 L1407.0,99.0 L1423.2,96.6 L1439.5,94.2 L1455.7,91.8 L1472.0,89.4 L1488.2,87.0
 L1504.5,84.7 L1520.8,82.4 L1537.0,80.1 L1553.3,77.8 L1569.5,75.6 L1585.8,73.3 L1602.0,71.1 L1618.3,69.0
 L1634.6,66.8 L1650.8,64.7 L1667.1,62.6 L1683.3,60.6 L1699.6,58.5 L1715.8,56.6 L1732.1,54.6 '/> 
 
 (0, 0)-WNLT 
 
 
 (0, 0)-WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' stroke-dasharray='7.5,12.0' d='M1526.1,108.1 L1579.5,108.1 M106.4,237.6 L138.9,237.5 L171.4,237.1 L203.9,236.4 L236.5,235.5 L269.0,234.4
 L301.5,233.0 L334.0,231.4 L366.5,229.5 L399.0,227.5 L431.5,225.1 L464.1,222.6 L496.6,219.8 L529.1,216.9
 L561.6,213.7 L594.1,210.3 L626.6,206.7 L659.1,203.0 L691.7,199.0 L724.2,195.0 L756.7,190.7 L789.2,186

In [ ]:
foreach(var lst in PlotTableSH[3]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case4/StartUpStudy/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

### Case 5

In [ ]:
var gp = PlotTableSH[4].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0003 
 
 
 
 
 -0.00025 
 
 
 
 
 -0.0002 
 
 
 
 
 -0.00015 
 
 
 
 
 -0.0001 
 
 
 
 
 -5x10 -5 
 
 
 
 
 0 
 
 
 
 
 5x10 -5 
 
 
 
 
 0.0001 
 
 
 
 
 0.00015 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M1526.1,60.1 L1579.5,60.1 M123.0,190.4 L139.1,185.1 L155.2,182.7 L171.3,181.2 L187.4,179.8 L203.5,178.4
 L219.5,176.7 L235.6,174.9 L251.7,172.9 L267.8,170.7 L283.9,168.4 L300.0,165.9 L316.1,163.4 L332.2,160.7
 L348.3,157.9 L364.4,155.1 L380.5,152.3 L396.5,149.4 L412.6,146.5 L428.7,143.6 L444.8,140.6 L460.9,137.7
 L477.0,134.9 L493.1,132.0 L509.2,129.2 L525.3,126.4 L541.4,123.7 L557.5,121.0 L573.5,118.3 L589.6,115.7
 L605.7,113.2 L621.8,110.7 L637.9,108.3 L654.0,105.9 L670.1,103.6 L686.2,101.4 L702.3,99.2 L718.4,97.1
 L734.5,95.0 L750.5,93.1 L766.6,91.1 L782.7,89.3 L798.8,87.5 L814.9,85.7 L831.0,84.1 L847.1,82.4
 L863.2,80.9 L879.3,79.4 L895.4,77.9 L911.5,76.6 L927.6,75.2 L943.6,73.9 L959.7,72.7 L975.8,71.5
 L991.9,70.3 L1008.0,69.2 L1024.1,68.2 L1040.2,67.2 L1056.3,66.2 L1072.4,65.2 L1088.5,64.3 L1104.6,63.5
 L1120.6,62.7 L1136.7,61.9 L1152.8,61.1 L1168.9,60.4 L1185.0,59.7 L1201.1,59.0 L1217.2,58.4 L1233.3,57.8
 L1249.4,57.2 L1265.5,56.6 L1281.6,56.1 L1297.6,55.6 L1313.7,55.1 L1329.8,54.6 L1345.9,54.1 L1362.0,53.7
 L1378.1,53.3 L1394.2,52.9 L1410.3,52.5 L1426.4,52.2 L1442.5,51.8 L1458.6,51.5 L1474.6,51.2 L1490.7,50.9
 L1506.8,50.6 L1522.9,50.3 L1539.0,50.1 L1555.1,49.8 L1571.2,49.6 L1587.3,49.4 L1603.4,49.2 L1619.5,49.0
 L1635.6,48.9 L1651.6,48.7 L1667.7,48.5 L1683.8,48.3 L1699.9,48.2 L1716.0,48.0 L1732.1,47.9 '/> 
 
 (0, 0)-Ainit 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,84.1 L1579.5,84.1 M123.0,190.4 L139.1,188.1 L155.2,188.6 L171.3,190.2 L187.4,191.7 L203.5,193.1
 L219.5,194.1 L235.6,194.8 L251.7,195.2 L267.8,195.3 L283.9,195.1 L300.0,194.7 L316.1,194.0 L332.2,193.2
 L348.3,192.2 L364.4,191.0 L380.5,189.8 L396.5,188.4 L412.6,187.0 L428.7,185.5 L444.8,183.9 L460.9,182.3
 L477.0,180.7 L493.1,179.1 L509.2,177.4 L525.3,175.8 L541.4,174.2 L557.5,172.6 L573.5,171.1 L589.6,169.6
 L605.7,168.1 L621.8,166.7 L637.9,165.3 L654.0,164.0 L670.1,162.7 L686.2,161.5 L702.3,160.3 L718.4,159.2
 L734.5,158.2 L750.5,157.2 L766.6,156.2 L782.7,155.3 L798.8,154.5 L814.9,153.7 L831.0,152.9 L847.1,152.2
 L863.2,151.6 L879.3,151.0 L895.4,150.4 L911.5,149.8 L927.6,149.3 L943.6,148.9 L959.7,148.4 L975.8,148.0
 L991.9,147.7 L1008.0,147.3 L1024.1,147.1 L1040.2,146.7 L1056.3,146.7 L1072.4,146.0 L1088.5,145.8 L1104.6,145.5
 L1120.6,145.2 L1136.7,145.0 L1152.8,144.8 L1168.9,144.6 L1185.0,144.4 L1201.1,144.2 L1217.2,144.0 L1233.3,143.8
 L1249.4,143.7 L1265.5,143.5 L1281.6,143.3 L1297.6,143.2 L1313.7,143.0 L1329.8,142.8 L1345.9,142.7 L1362.0,142.5
 L1378.1,142.3 L1394.2,142.2 L1410.3,142.0 L1426.4,141.8 L1442.5,141.6 L1458.6,141.4 L1474.6,141.2 L1490.7,141.0
 L1506.8,140.8 L1522.9,140.6 L1539.0,140.4 L1555.1,140.2 L1571.2,139.9 L1587.3,139.7 L1603.4,139.4 L1619.5,139.1
 L1635.6,168.3 L1651.6,147.3 L1667.7,168.4 L1683.8,167.5 L1699.9,146.4 L1716.0,140.5 L1732.1,138.1 '/> 
 
 (0, 0)-WNLT 
 
 
 (0, 0)-WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' stroke-dasharray='7.5,12.0' d='M1526.1,108.1 L1579.5,108.1 M123.0,250.5 L155.2,248.5 L187.4,245.5 L219.5,241.5 L251.7,236.9 L283.9,231.7
 L316.1,226.2 L348.3,220.4 L380.5,214.5 L412.6,208.5 L444.8,202.6 L477.0,196.7 L509.2,191.0 L541.4,185.5
 L573.5,180.2 L605.7,175.2 L637.9,170.4 L670.1,165.8 L702.3,161.5 L734.5,157.5 L766.6,153

In [ ]:
foreach(var lst in PlotTableSH[4]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case5/StartUpStudy/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

### All-in-one plot

In [48]:
// for(int i = 0; i < PlotTableSH.GetLength(0); i++) {
//    PlotTableSH[i,0].XrangeMin = 0;
//    PlotTableSH[i,0].XrangeMax = 2.5;
// }

In [ ]:
// var gp = PlotTableSH.ToGnuplot();
// gp.PlotSVG(xRes:1800,yRes:1500)

In [ ]:
//Note: you must create sub-directory "latex" first!
//var cl = gp.PlotCairolatex(xSize:32,ySize:18);
//cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}

# Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};
double surfEnrgy0 = Math.PI;    // surface energy for a sphere with r = 1 and sigma = 1 on a quarter domain

In [ ]:
Plot2Ddata[,] PlotTableEnrgy = new Plot2Ddata[caseS.Length, 2];
for(int iRow = 0; iRow < PlotTableEnrgy.GetLength(0); iRow++) { // loop over cases
for(int iCol = 0; iCol < PlotTableEnrgy.GetLength(1); iCol++) {
for(int iAMR = 0; iAMR < amrS.Length; iAMR++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
    string _amr = amrS[iAMR];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTableEnrgy[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    if(iCol == 0) {
        {
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines; 
            fmt.LineWidth = 3;
            fmt.LineColor = LineColors.Blue;
            if (useNewton[iNew]) 
                fmt.DashType = DashTypes.Dashed;
            else  
                fmt.DashType = DashTypes.Solid;          

            string name = "surfaceEnergy";
            if(iAMR > 0)
                name = name + "-amr";
            if(ShouldAnaInit[iAna])
                name = name + "-Ainit";
            if(useNewton[iNew])
                name = name + "-Newton";

            List<double> procSurfEnrgy = new List<double>();    
            foreach(double sEnrgyVal in tab["surfaceEnergy"]) {
                procSurfEnrgy.Add(sEnrgyVal - surfEnrgy0);
            }    
            
            plot.AddDataGroup(name, time, procSurfEnrgy, fmt);
        }
        {
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines; 
            fmt.LineWidth = 3;
            fmt.LineColor = LineColors.Red;
            if (useNewton[iNew]) 
                fmt.DashType = DashTypes.Dashed;
            else  
                fmt.DashType = DashTypes.Solid;          

            string name = "kineticEnergy";
            if(iAMR > 0)
                name = name + "-amr";
            if(ShouldAnaInit[iAna])
                name = name + "-Ainit";
            if(useNewton[iNew])
                name = name + "-Newton";
            
            plot.AddDataGroup(name, time, tab["kineticEnergy"], fmt);
        }
    }

    if(iCol == 1) {

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines; 
        fmt.LineWidth = 3;
        fmt.LineColor = LineColors.Blue;
        if (useNewton[iNew]) 
            fmt.DashType = DashTypes.Dashed;
        else  
            fmt.DashType = DashTypes.Solid;

        string name = "totalEnergy";
        if(iAMR > 0)
            name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";
        
        plot.AddDataGroup(name, time, tab["totalEnergy"], fmt);
    }
    
   
    //plot.ShowLegend = iCol == 2 && iRow == 2;
    plot.ShowLegend = true;
    
    //plot.Title = SI.Name;
        
    if(iAMR == 0) {
        PlotTableEnrgy[iRow,iCol] = plot;
    } else {
        PlotTableEnrgy[iRow,iCol] = PlotTableEnrgy[iRow,iCol].Merge(plot);
    }
        
    PlotTableEnrgy[iRow,iCol].XrangeMin = 0;
    PlotTableEnrgy[iRow,iCol].XrangeMax = 4;
}
}
}
}
}

J432k3_amr0_case1_Oh0.1_AnalytInit
J432k3_amr0_case1_Oh0.1_AnalytInit
J432k3_amr0_case2_Oh0.1_AnalytInit
J432k3_amr0_case2_Oh0.1_AnalytInit
J432k3_amr0_case3_Oh0.1_AnalytInit
J432k3_amr0_case3_Oh0.1_AnalytInit
J432k3_amr0_case4_Oh0.1_AnalytInit
J432k3_amr0_case4_Oh0.1_AnalytInit
J432k3_amr0_case5_Oh0.56_AnalytInit
J432k3_amr0_case5_Oh0.56_AnalytInit


## Case1

In [ ]:
var gp = PlotTableEnrgy[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M53.9,438.6 L54.8,437.7 L55.6,435.4 L56.5,432.7 L57.4,429.7 L58.3,426.8
 L59.1,423.4 L60.0,420.1 L60.9,417.0 L61.7,414.0 L62.6,411.4 L63.5,408.9 L64.3,406.7 L65.2,417.8
 L66.1,416.1 L67.0,414.7 L67.8,413.5 L68.7,412.7 L69.6,411.9 L70.4,413.2 L71.3,412.7 L72.2,412.3
 L73.0,412.1 L73.9,413.5 L74.8,413.6 L75.7,413.6 L76.5,413.7 L77.4,413.8 L78.3,414.2 L79.1,414.9
 L80.0,412.6 L80.9,413.1 L81.7,409.9 L82.6,410.6 L83.5,411.2 L84.4,412.7 L85.2,413.3 L86.1,413.8
 L87.0,407.3 L87.8,407.3 L88.7,407.5 L89.6,407.7 L90.5,407.9 L91.3,407.9 L92.2,411.4 L93.1,416.2
 L93.9,419.0 L94.8,405.9 L95.7,407.0 L96.5,409.6 L97.4,409.6 L98.3,410.7 L99.2,410.8 L100.0,415.6
 L100.9,406.5 L101.8,401.4 L102.6,401.4 L103.5,401.6 L104.4,401.7 L105.2,401.8 L106.1,401.8 L107.0,401.8
 L107.9,401.8 L108.7,401.7 L109.6,402.0 L110.5,402.3 L111.3,402.5 L112.2,402.7 L113.1,402.8 L113.9,403.0
 L114.8,403.1 L115.7,403.2 L116.6,403.4 L117.4,403.7 L118.3,403.9 L119.2,404.2 L120.0,404.9 L120.9,405.3
 L121.8,405.8 L122.6,406.3 L123.5,406.8 L124.4,407.4 L125.3,408.6 L126.1,409.3 L127.0,408.7 L127.9,410.1
 L128.7,411.0 L129.6,411.8 L130.5,412.6 L131.4,411.9 L132.2,412.8 L133.1,412.1 L134.0,413.0 L134.8,414.0
 L135.7,414.9 L136.6,415.8 L137.4,416.7 L138.3,411.3 L139.2,412.1 L140.1,413.0 L140.9,413.9 L141.8,414.7
 L142.7,415.7 L143.5,416.6 L144.4,417.5 L145.3,418.3 L146.1,419.3 L147.0,420.3 L147.9,421.3 L148.8,422.3
 L149.6,423.2 L150.5,424.1 L151.4,425.4 L152.2,426.4 L153.1,427.3 L154.0,428.2 L154.8,429.1 L155.7,430.1
 L156.6,431.1 L157.5,432.1 L158.3,433.1 L159.2,434.1 L160.1,435.4 L160.9,436.4 L161.8,437.2 L162.7,438.1
 L163.6,439.0 L164.4,440.0 L165.3,440.9 L166.2,441.8 L167.0,442.7 L167.9,443.6 L168.8,444.4 L169.6,445.3
 L170.5,446.1 L171.4,447.0 L172.3,447.8 L173.1,448.6 L174.0,449.3 L174.9,450.1 L175.7,450.8 L176.6,451.5
 L177.5,452.2 L178.3,452.9 L179.2,453.6 L180.1,454.2 L181.0,454.9 L181.8,455.5 L182.7,456.1 L183.6,456.7
 L184.4,457.3 L185.3,457.8 L186.2,458.4 L187.0,458.9 L187.9,459.5 L188.8,460.0 L189.7,460.5 L190.5,461.0
 L191.4,461.5 L192.3,462.0 L193.1,462.5 L194.0,463.0 L194.9,463.5 L195.8,464.0 L196.6,464.5 L197.5,465.0
 L198.4,465.5 L199.2,466.0 L200.1,466.6 L201.0,467.1 L201.8,467.6 L202.7,468.1 L203.6,468.7 L204.5,469.2
 L205.3,469.8 L206.2,470.4 L207.1,470.9 L207.9,471.5 L208.8,472.1 L209.7,472.7 L210.5,473.3 L211.4,473.9
 L212.3,474.5 L213.2,475.2 L214.0,475.8 L214.9,476.4 L215.8,477.1 L216.6,477.7 L217.5,478.4 L218.4,479.0
 L219.2,479.7 L220.1,480.3 L221.0,481.0 L221.9,481.7 L222.7,482.7 L223.6,483.3 L224.5,483.9 L225.3,484.4
 L226.2,485.0 L227.1,485.6 L228.0,486.2 L228.8,486.8 L229.7,487.4 L230.6,488.0 L231.4,488.6 L232.3,489.1
 L233.2,489.7 L234.0,490.2 L234.9,490.7 L235.8,491.3 L236.7,491.7 L237.5,492.2 L238.4,492.7 L239.3,493.1
 L240.1,493.5 L241.0,493.9 L241.9,494.3 L242.7,494.7 L243.6,495.0 L244.5,495.3 L245.4,495.6 L246.2,495.9
 L247.1,496.1 L248.0,496.4 L248.8,496.6 L249.7,496.8 L250.6,496.9 L251.4,497.0 L252.3,497.2 L253.2,497.3
 L254.1,497.3 L254.9,497.4 L255.8,497.4 L256.7,497.9 L257.5,497.8 L258.4,497.8 L259.3,497.8 L260.1,497.7
 L261.0,497.7 L261.9,497.5 L262.8,497.4 L263.6,497.3 L264.5,497.1 L265.4,496.9 L266.2,497.0 L267.1,496.8
 L268.0

In [ ]:
var gp = PlotTableEnrgy[0,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 3.2 
 
 
 
 
 3.4 
 
 
 
 
 3.6 
 
 
 
 
 3.8 
 
 
 
 
 4 
 
 
 
 
 4.2 
 
 
 
 
 4.4 
 
 
 
 
 4.6 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M53.9,60.5 L54.8,169.1 L55.6,220.0 L56.5,254.9 L57.4,280.5 L58.3,299.7
 L59.1,314.5 L60.0,326.1 L60.9,335.2 L61.7,342.5 L62.6,348.5 L63.5,353.5 L64.3,357.7 L65.2,374.4
 L66.1,377.7 L67.0,380.6 L67.8,383.3 L68.7,386.0 L69.6,388.3 L70.4,392.5 L71.3,394.5 L72.2,396.4
 L73.0,398.2 L73.9,401.6 L74.8,403.5 L75.7,405.1 L76.5,406.7 L77.4,408.1 L78.3,409.8 L79.1,411.7
 L80.0,410.6 L80.9,412.1 L81.7,409.8 L82.6,411.4 L83.5,412.8 L84.4,415.1 L85.2,416.4 L86.1,417.5
 L87.0,411.6 L87.8,412.1 L88.7,412.8 L89.6,413.6 L90.5,414.2 L91.3,414.6 L92.2,418.4 L93.1,423.6
 L93.9,426.8 L94.8,414.0 L95.7,415.4 L96.5,418.3 L97.4,418.6 L98.3,419.9 L99.2,420.2 L100.0,425.3
 L100.9,416.4 L101.8,411.6 L102.6,411.8 L103.5,412.1 L104.4,412.5 L105.2,412.7 L106.1,412.9 L107.0,413.0
 L107.9,413.1 L108.7,413.2 L109.6,413.6 L110.5,414.0 L111.3,414.3 L112.2,414.6 L113.1,414.8 L113.9,415.0
 L114.8,415.2 L115.7,415.3 L116.6,415.5 L117.4,415.7 L118.3,415.9 L119.2,416.1 L120.0,416.7 L120.9,417.0
 L121.8,417.4 L122.6,417.7 L123.5,418.1 L124.4,418.5 L125.3,419.5 L126.1,419.9 L127.0,419.0 L127.9,420.2
 L128.7,420.7 L129.6,421.2 L130.5,421.6 L131.4,420.6 L132.2,421.1 L133.1,420.0 L134.0,420.5 L134.8,420.9
 L135.7,421.4 L136.6,421.9 L137.4,422.3 L138.3,416.4 L139.2,416.7 L140.1,417.1 L140.9,417.4 L141.8,417.8
 L142.7,418.2 L143.5,418.6 L144.4,418.9 L145.3,419.2 L146.1,419.6 L147.0,420.0 L147.9,420.5 L148.8,421.0
 L149.6,421.4 L150.5,421.8 L151.4,422.5 L152.2,423.0 L153.1,423.4 L154.0,423.7 L154.8,424.2 L155.7,424.7
 L156.6,425.2 L157.5,425.8 L158.3,426.3 L159.2,426.9 L160.1,427.8 L160.9,428.3 L161.8,428.8 L162.7,429.3
 L163.6,429.9 L164.4,430.5 L165.3,431.1 L166.2,431.7 L167.0,432.3 L167.9,432.9 L168.8,433.5 L169.6,434.1
 L170.5,434.7 L171.4,435.3 L172.3,435.9 L173.1,436.6 L174.0,437.2 L174.9,437.8 L175.7,438.3 L176.6,438.9
 L177.5,439.5 L178.3,440.1 L179.2,440.7 L180.1,441.2 L181.0,441.8 L181.8,442.3 L182.7,442.8 L183.6,443.4
 L184.4,443.9 L185.3,444.4 L186.2,444.9 L187.0,445.4 L187.9,445.9 L188.8,446.3 L189.7,446.8 L190.5,447.3
 L191.4,447.7 L192.3,448.1 L193.1,448.6 L194.0,449.0 L194.9,449.4 L195.8,449.9 L196.6,450.3 L197.5,450.7
 L198.4,451.1 L199.2,451.5 L200.1,451.9 L201.0,452.2 L201.8,452.6 L202.7,453.0 L203.6,453.4 L204.5,453.8
 L205.3,454.2 L206.2,454.5 L207.1,454.9 L207.9,455.3 L208.8,455.7 L209.7,456.1 L210.5,456.5 L211.4,456.8
 L212.3,457.2 L213.2,457.6 L214.0,458.0 L214.9,458.4 L215.8,458.8 L216.6,459.2 L217.5,459.6 L218.4,460.0
 L219.2,460.4 L220.1,460.9 L221.0,461.3 L221.9,461.7 L222.7,462.5 L223.6,462.9 L224.5,463.3 L225.3,463.6
 L226.2,464.0 L227.1,464.4 L228.0,464.8 L228.8,465.3 L229.7,465.7 L230.6,466.2 L231.4,466.6 L232.3,467.1
 L233.2,467.5 L234.0,468.0 L234.9,468.5 L235.8,469.0 L236.7,469.4 L237.5,469.9 L238.4,470.4 L239.3,470.9
 L240.1,471.4 L241.0,471.8 L241.9,472.3 L242.7,472.8 L243.6,473.3 L244.5,473.7 L245.4,474.2 L246.2,474.7
 L247.1,475.1 L248.0,475.6 L248.8,476.1 L249.7,476.5 L250.6,477.0 L251.4,477.4 L252.3,477.8 L253.2,478.3
 L254.1,478.7 L254.9,479.1 L255.8,479.5 L256.7,480.4 L257.5,480.7 L258.4,481.1 L259.3,481.5 L260.1,481.9
 L261.0,482.4 L261.9,482.6 L262.8,483.0 L263.6,483.4 L264.5,483.6 L265.4,483.9 L266.2,484.5 L267.1,484.8
 L268.0,485.2

## Case 2

In [ ]:
var gp = PlotTableEnrgy[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M45.6,475.8 L46.0,474.8 L46.5,474.0 L46.9,473.7 L47.4,473.1 L47.8,472.6
 L48.2,472.1 L48.7,471.7 L49.1,471.4 L49.6,471.0 L50.0,470.7 L50.4,470.6 L50.9,470.3 L51.3,470.6
 L51.8,469.2 L52.2,469.0 L52.6,468.8 L53.1,468.7 L53.5,468.5 L54.0,468.3 L54.4,468.2 L54.8,468.1
 L55.3,468.0 L55.7,467.9 L56.2,467.8 L56.6,467.7 L57.0,467.7 L57.5,467.7 L57.9,467.6 L58.4,467.6
 L58.8,467.7 L59.2,467.7 L59.7,467.8 L60.1,468.2 L60.6,468.7 L61.0,468.8 L61.5,468.5 L61.9,466.3
 L62.3,468.9 L62.8,468.6 L63.2,468.6 L63.7,468.9 L64.1,468.1 L64.5,468.3 L65.0,468.5 L65.4,468.7
 L65.9,468.8 L66.3,469.0 L66.7,469.1 L67.2,469.2 L67.6,469.3 L68.1,469.4 L68.5,469.5 L68.9,469.6
 L69.4,469.7 L69.8,469.8 L70.3,469.8 L70.7,469.9 L71.1,469.9 L71.6,470.0 L72.0,470.1 L72.5,470.2
 L72.9,470.2 L73.3,470.3 L73.8,470.4 L74.2,470.5 L74.7,470.6 L75.1,470.6 L75.5,470.7 L76.0,470.8
 L76.4,470.9 L76.9,470.3 L77.3,470.5 L77.7,470.6 L78.2,470.6 L78.6,470.7 L79.1,470.1 L79.5,471.4
 L79.9,471.4 L80.4,471.2 L80.8,471.1 L81.3,470.6 L81.7,471.9 L82.1,471.9 L82.6,471.9 L83.0,471.9
 L83.5,471.9 L83.9,471.9 L84.3,472.0 L84.8,472.0 L85.2,471.9 L85.7,471.9 L86.1,471.7 L86.5,471.2
 L87.0,472.4 L87.4,472.4 L87.9,472.4 L88.3,472.2 L88.8,472.1 L89.2,472.0 L89.6,472.0 L90.1,472.1
 L90.5,472.1 L91.0,472.1 L91.4,472.1 L91.8,472.1 L92.3,472.1 L92.7,472.2 L93.2,472.1 L93.6,472.2
 L94.0,472.1 L94.5,472.1 L94.9,472.2 L95.4,472.1 L95.8,472.1 L96.2,472.0 L96.7,471.9 L97.1,472.0
 L97.6,471.7 L98.0,471.8 L98.4,471.6 L98.9,472.3 L99.3,472.2 L99.8,472.2 L100.2,472.1 L100.6,472.0
 L101.1,471.0 L101.5,472.3 L102.0,471.8 L102.4,472.0 L102.8,472.6 L103.3,472.5 L103.7,471.5 L104.2,472.1
 L104.6,473.4 L105.0,473.5 L105.5,472.6 L105.9,473.3 L106.4,472.5 L106.8,472.9 L107.2,473.0 L107.7,472.0
 L108.1,473.4 L108.6,473.0 L109.0,473.7 L109.4,473.3 L109.9,473.4 L110.3,472.9 L110.8,472.0 L111.2,474.1
 L111.6,474.1 L112.1,474.2 L112.5,474.4 L113.0,474.4 L113.4,474.2 L113.8,474.8 L114.3,474.6 L114.7,474.6
 L115.2,474.2 L115.6,473.9 L116.1,473.8 L116.5,474.5 L116.9,474.8 L117.4,474.8 L117.8,475.2 L118.3,475.0
 L118.7,475.1 L119.1,474.9 L119.6,475.0 L120.0,474.8 L120.5,474.6 L120.9,474.2 L121.3,474.2 L121.8,474.0
 L122.2,473.2 L122.7,472.8 L123.1,474.4 L123.5,472.6 L124.0,475.2 L124.4,475.0 L124.9,475.2 L125.3,475.1
 L125.7,474.8 L126.2,474.9 L126.6,474.6 L127.1,474.4 L127.5,474.3 L127.9,475.3 L128.4,475.4 L128.8,475.4
 L129.3,475.5 L129.7,475.4 L130.1,475.3 L130.6,475.3 L131.0,475.2 L131.5,475.2 L131.9,475.1 L132.3,475.1
 L132.8,475.1 L133.2,475.0 L133.7,475.1 L134.1,475.1 L134.5,475.1 L135.0,475.1 L135.4,475.2 L135.9,475.2
 L136.3,475.3 L136.7,475.3 L137.2,475.3 L137.6,475.4 L138.1,475.4 L138.5,475.4 L138.9,475.5 L139.4,475.5
 L139.8,475.6 L140.3,475.6 L140.7,475.7 L141.1,475.7 L141.6,475.7 L142.0,475.8 L142.5,475.8 L142.9,475.9
 L143.3,475.9 L143.8,476.0 L144.2,476.0 L144.7,476.0 L145.1,476.1 L145.6,476.1 L146.0,476.2 L146.4,476.3
 L146.9,476.3 L147.3,476.3 L147.8,476.4 L148.2,476.4 L148.6,476.4 L149.1,476.6 L149.5,476.6 L150.0,476.6
 L150.4,476.7 L150.8,476.7 L151.3,476.7 L151.7,476.8 L152.2,476.9 L152.6,476.9 L153.0,477.0 L153.5,477.0
 L153.9,477.0 L154.4,477.1 L154.8,477.1 L155.2,477.1 L155.7,477.2 L156.1,477.2 L156.6,477.3 L157.0,477.3
 L157.4

In [ ]:
var gp = PlotTableEnrgy[1,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M45.6,57.8 L46.0,258.7 L46.5,327.9 L46.9,363.2 L47.4,382.9 L47.8,396.5
 L48.2,407.7 L48.7,417.8 L49.1,424.1 L49.6,429.6 L50.0,435.5 L50.4,438.8 L50.9,441.6 L51.3,439.6
 L51.8,448.7 L52.2,452.0 L52.6,454.2 L53.1,456.2 L53.5,450.7 L54.0,459.5 L54.4,462.2 L54.8,464.0
 L55.3,465.6 L55.7,467.1 L56.2,468.5 L56.6,469.8 L57.0,471.2 L57.5,472.5 L57.9,473.8 L58.4,475.1
 L58.8,476.2 L59.2,477.0 L59.7,478.6 L60.1,480.1 L60.6,477.7 L61.0,481.1 L61.5,482.2 L61.9,480.5
 L62.3,483.2 L62.8,482.5 L63.2,482.0 L63.7,484.5 L64.1,485.9 L64.5,487.8 L65.0,489.2 L65.4,490.3
 L65.9,491.2 L66.3,492.0 L66.7,492.7 L67.2,493.3 L67.6,493.8 L68.1,494.3 L68.5,494.7 L68.9,495.1
 L69.4,495.5 L69.8,495.7 L70.3,496.0 L70.7,496.2 L71.1,496.4 L71.6,496.7 L72.0,496.9 L72.5,497.1
 L72.9,497.3 L73.3,497.5 L73.8,497.7 L74.2,497.8 L74.7,498.0 L75.1,498.1 L75.5,498.3 L76.0,498.4
 L76.4,498.6 L76.9,498.1 L77.3,498.2 L77.7,498.5 L78.2,498.5 L78.6,498.6 L79.1,498.1 L79.5,499.4
 L79.9,499.4 L80.4,499.3 L80.8,499.1 L81.3,498.6 L81.7,500.0 L82.1,500.1 L82.6,500.1 L83.0,500.1
 L83.5,500.1 L83.9,500.1 L84.3,500.2 L84.8,500.2 L85.2,500.2 L85.7,500.1 L86.1,500.0 L86.5,499.5
 L87.0,500.7 L87.4,500.7 L87.9,500.7 L88.3,500.5 L88.8,500.4 L89.2,500.4 L89.6,500.4 L90.1,500.4
 L90.5,500.4 L91.0,500.5 L91.4,500.5 L91.8,500.5 L92.3,500.5 L92.7,500.6 L93.2,500.6 L93.6,500.7
 L94.0,500.6 L94.5,500.6 L94.9,500.7 L95.4,500.6 L95.8,500.7 L96.2,500.5 L96.7,500.5 L97.1,500.6
 L97.6,500.3 L98.0,500.4 L98.4,500.2 L98.9,501.0 L99.3,500.9 L99.8,500.9 L100.2,500.8 L100.6,500.7
 L101.1,499.8 L101.5,501.0 L102.0,500.6 L102.4,500.8 L102.8,501.4 L103.3,501.4 L103.7,500.4 L104.2,501.0
 L104.6,502.3 L105.0,502.4 L105.5,501.6 L105.9,502.2 L106.4,501.4 L106.8,501.9 L107.2,502.0 L107.7,501.0
 L108.1,502.5 L108.6,502.0 L109.0,502.7 L109.4,502.4 L109.9,502.5 L110.3,502.0 L110.8,501.1 L111.2,503.3
 L111.6,503.2 L112.1,503.4 L112.5,503.7 L113.0,503.6 L113.4,503.4 L113.8,504.0 L114.3,503.9 L114.7,503.8
 L115.2,503.5 L115.6,503.2 L116.1,503.1 L116.5,503.8 L116.9,504.2 L117.4,504.1 L117.8,504.6 L118.3,504.4
 L118.7,504.5 L119.1,504.3 L119.6,504.3 L120.0,504.2 L120.5,504.0 L120.9,503.6 L121.3,503.7 L121.8,503.4
 L122.2,502.7 L122.7,502.2 L123.1,503.8 L123.5,502.1 L124.0,504.7 L124.4,504.5 L124.9,504.7 L125.3,504.5
 L125.7,504.3 L126.2,504.3 L126.6,504.1 L127.1,503.8 L127.5,503.8 L127.9,504.8 L128.4,504.9 L128.8,504.8
 L129.3,505.0 L129.7,504.9 L130.1,504.8 L130.6,504.8 L131.0,504.7 L131.5,504.6 L131.9,504.6 L132.3,504.6
 L132.8,504.5 L133.2,504.5 L133.7,504.5 L134.1,504.5 L134.5,504.6 L135.0,504.6 L135.4,504.6 L135.9,504.7
 L136.3,504.7 L136.7,504.7 L137.2,504.8 L137.6,504.8 L138.1,504.9 L138.5,504.9 L138.9,504.9 L139.4,505.0
 L139.8,505.0 L140.3,505.1 L140.7,505.1 L141.1,505.1 L141.6,505.2 L142.0,505.2 L142.5,505.2 L142.9,505.3
 L143.3,505.3 L143.8,505.4 L144.2,505.4 L144.7,505.5 L145.1,505.5 L145.6,505.5 L146.0,505.6 L146.4,505.7
 L146.9,505.7 L147.3,505.8 L147.8,505.8 L148.2,505.8 L148.6,505.9 L149.1,506.0 L149.5,506.0 L150.0,506.1
 L150.4,506.1 L150.8,506.1 L151.3,506.2 L151.7,506.2 L152.2,506.3 L152.6,506.4 L153.0,506.4 L153.5,506.5
 L153.9,506.5 L154.4,506.5 L154.8,506.6 L155.2,506.6 L155.7,506.7 L156.1,506.7 L156.6,506.8 L157.0,506.8
 L157.4,506

## Case 3

In [ ]:
var gp = PlotTableEnrgy[2,0].ToGnuplot();
gp.PlotSVG()

## Case 4

In [ ]:
var gp = PlotTableEnrgy[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0.04 
 
 
 
 
 0.045 
 
 
 
 
 0.05 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M70.5,55.6 L71.3,54.1 L72.2,53.2 L73.0,52.9 L73.9,52.8 L74.7,53.0
 L75.6,53.4 L76.4,54.0 L77.3,54.7 L78.1,55.6 L79.0,56.7 L79.8,57.8 L80.7,59.1 L81.5,60.5
 L82.4,62.1 L83.2,63.8 L84.1,65.5 L84.9,67.4 L85.8,69.5 L86.6,71.6 L87.5,73.8 L88.3,76.2
 L89.2,78.6 L90.0,81.2 L90.9,83.9 L91.7,86.6 L92.6,89.5 L93.4,92.4 L94.3,95.5 L95.1,98.6
 L96.0,101.8 L96.8,105.1 L97.7,108.5 L98.5,111.9 L99.4,115.5 L100.2,119.1 L101.1,122.7 L101.9,126.5
 L102.8,130.3 L103.6,134.2 L104.5,138.1 L105.3,142.2 L106.2,146.3 L107.0,150.4 L107.9,154.6 L108.7,158.9
 L109.6,163.3 L110.4,167.7 L111.3,172.4 L112.1,176.9 L113.0,181.4 L113.8,185.9 L114.7,193.0 L115.5,198.2
 L116.4,201.8 L117.2,205.8 L118.1,210.2 L118.9,214.8 L119.8,219.5 L120.6,224.3 L121.5,229.2 L122.3,234.1
 L123.2,239.0 L124.0,244.0 L124.9,248.9 L125.7,254.0 L126.6,259.0 L127.4,264.0 L128.3,269.1 L129.1,274.1
 L130.0,279.2 L130.8,284.3 L131.7,289.4 L132.5,294.4 L133.4,299.5 L134.2,304.5 L135.1,309.6 L135.9,314.6
 L136.8,319.6 L137.6,324.6 L138.5,329.6 L139.3,334.6 L140.2,339.5 L141.0,344.4 L141.9,349.3 L142.7,354.2
 L143.6,359.0 L144.4,363.8 L145.3,368.5 L146.1,373.2 L147.0,377.9 L147.8,382.5 L148.7,387.1 L149.5,391.6
 L150.4,396.1 L151.2,400.5 L152.1,404.9 L152.9,409.2 L153.8,413.4 L154.6,417.6 L155.5,421.7 L156.3,425.8
 L157.1,429.8 L158.0,433.7 L158.8,437.5 L159.7,441.3 L160.5,445.0 L161.4,448.6 L162.2,452.1 L163.1,455.5
 L163.9,458.9 L164.8,462.2 L165.6,465.3 L166.5,468.4 L167.3,471.4 L168.2,474.3 L169.0,477.2 L169.9,480.0
 L170.7,482.6 L171.6,485.2 L172.4,487.7 L173.3,490.1 L174.1,492.3 L175.0,494.5 L175.8,496.6 L176.7,498.5
 L177.5,500.4 L178.4,502.2 L179.2,503.8 L180.1,505.2 L180.9,506.7 L181.8,508.1 L182.6,509.3 L183.5,510.4
 L184.3,511.4 L185.2,512.3 L186.0,513.1 L186.9,513.8 L187.7,514.4 L188.6,520.4 L189.4,519.0 L190.3,518.2
 L191.1,517.9 L192.0,517.7 L192.8,517.5 L193.7,517.3 L194.5,517.0 L195.4,516.7 L196.2,520.3 L197.1,518.0
 L197.9,516.4 L198.8,515.1 L199.6,514.1 L200.5,513.1 L201.3,512.1 L202.2,511.0 L203.0,509.9 L203.9,508.6
 L204.7,507.4 L205.6,506.0 L206.4,504.6 L207.3,503.2 L208.1,501.6 L209.0,500.0 L209.8,496.6 L210.7,494.2
 L211.5,492.0 L212.4,490.0 L213.2,487.9 L214.1,486.2 L214.9,484.1 L215.8,482.1 L216.6,480.0 L217.5,477.9
 L218.3,475.8 L219.2,473.6 L220.0,471.4 L220.9,469.1 L221.7,466.8 L222.6,464.4 L223.4,462.1 L224.3,459.7
 L225.1,457.2 L226.0,454.8 L226.8,452.3 L227.7,449.8 L228.5,447.3 L229.4,444.7 L230.2,442.0 L231.1,439.4
 L231.9,436.8 L232.8,434.0 L233.6,431.3 L234.5,428.5 L235.3,425.7 L236.2,422.4 L237.0,419.6 L237.9,416.9
 L238.7,414.2 L239.6,411.6 L240.4,409.0 L241.2,406.3 L242.1,403.4 L242.9,400.7 L243.8,398.0 L244.6,395.4
 L245.5,392.7 L246.3,390.1 L247.2,387.5 L248.0,384.9 L248.9,382.4 L249.7,380.1 L250.6,377.9 L251.4,375.7
 L252.3,373.7 L253.1,371.9 L254.0,370.1 L254.8,368.3 L255.7,366.5 L256.5,364.3 L257.4,362.1 L258.2,360.2
 L259.1,358.4 L259.9,356.7 L260.8,355.2 L261.6,353.8 L262.5,352.5 L263.3,351.3 L264.2,350.2 L265.0,349.3
 L265.9,348.4 L266.7,347.6 L267.6,346.9 L268.4,346.3 L269.3,345.7 L270.1,345.3 L271.0,344.9 L271.8,344.6
 L272.7,344.4 L273.5,344.2 L274.4

In [ ]:
var gp = PlotTableEnrgy[3,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.145 
 
 
 
 
 3.15 
 
 
 
 
 3.155 
 
 
 
 
 3.16 
 
 
 
 
 3.165 
 
 
 
 
 3.17 
 
 
 
 
 3.175 
 
 
 
 
 3.18 
 
 
 
 
 3.185 
 
 
 
 
 3.19 
 
 
 
 
 3.195 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M70.5,81.2 L71.3,83.4 L72.2,83.6 L73.0,83.8 L73.9,84.1 L74.7,84.3
 L75.6,84.6 L76.4,84.9 L77.3,85.2 L78.1,85.5 L79.0,85.9 L79.8,86.2 L80.7,86.5 L81.5,86.8
 L82.4,87.1 L83.2,87.5 L84.1,87.8 L84.9,88.2 L85.8,88.6 L86.6,89.0 L87.5,89.4 L88.3,89.9
 L89.2,90.3 L90.0,90.8 L90.9,91.3 L91.7,91.8 L92.6,92.4 L93.4,92.9 L94.3,93.5 L95.1,94.1
 L96.0,94.7 L96.8,95.4 L97.7,96.1 L98.5,96.8 L99.4,97.5 L100.2,98.2 L101.1,99.0 L101.9,99.8
 L102.8,100.6 L103.6,101.4 L104.5,102.2 L105.3,103.1 L106.2,104.0 L107.0,104.9 L107.9,105.9 L108.7,106.9
 L109.6,107.9 L110.4,109.1 L111.3,110.3 L112.1,111.5 L113.0,112.5 L113.8,113.6 L114.7,117.5 L115.5,119.2
 L116.4,119.2 L117.2,119.7 L118.1,120.5 L118.9,121.5 L119.8,122.7 L120.6,124.0 L121.5,125.3 L122.3,126.7
 L123.2,128.1 L124.0,129.5 L124.9,131.0 L125.7,132.5 L126.6,134.0 L127.4,135.6 L128.3,137.2 L129.1,138.9
 L130.0,140.5 L130.8,142.2 L131.7,144.0 L132.5,145.7 L133.4,147.5 L134.2,149.3 L135.1,151.2 L135.9,153.0
 L136.8,154.9 L137.6,156.9 L138.5,158.8 L139.3,160.8 L140.2,162.9 L141.0,164.9 L141.9,167.0 L142.7,169.1
 L143.6,171.3 L144.4,173.5 L145.3,175.7 L146.1,178.0 L147.0,180.2 L147.8,182.6 L148.7,184.9 L149.5,187.3
 L150.4,189.6 L151.2,192.1 L152.1,194.5 L152.9,197.0 L153.8,199.5 L154.6,202.0 L155.5,204.5 L156.3,207.1
 L157.1,209.7 L158.0,212.3 L158.8,215.0 L159.7,217.6 L160.5,220.3 L161.4,223.0 L162.2,225.7 L163.1,228.4
 L163.9,231.1 L164.8,233.9 L165.6,236.6 L166.5,239.4 L167.3,242.2 L168.2,245.0 L169.0,247.8 L169.9,250.7
 L170.7,253.5 L171.6,256.4 L172.4,259.2 L173.3,262.1 L174.1,265.0 L175.0,267.8 L175.8,270.7 L176.7,273.6
 L177.5,276.4 L178.4,279.2 L179.2,282.1 L180.1,284.7 L180.9,287.6 L181.8,290.4 L182.6,293.2 L183.5,296.0
 L184.3,298.7 L185.2,301.4 L186.0,304.2 L186.9,306.9 L187.7,309.6 L188.6,318.2 L189.4,318.9 L190.3,320.3
 L191.1,322.3 L192.0,324.5 L192.8,326.9 L193.7,329.3 L194.5,331.7 L195.4,334.1 L196.2,340.9 L197.1,341.3
 L197.9,342.5 L198.8,344.2 L199.6,346.2 L200.5,348.3 L201.3,350.5 L202.2,352.6 L203.0,354.7 L203.9,356.8
 L204.7,358.9 L205.6,360.9 L206.4,363.0 L207.3,364.9 L208.1,366.9 L209.0,327.7 L209.8,346.5 L210.7,358.5
 L211.5,365.3 L212.4,369.4 L213.2,372.2 L214.1,374.8 L214.9,376.9 L215.8,378.7 L216.6,380.5 L217.5,382.3
 L218.3,384.0 L219.2,385.6 L220.0,387.1 L220.9,388.6 L221.7,390.0 L222.6,391.4 L223.4,392.8 L224.3,394.1
 L225.1,395.4 L226.0,396.6 L226.8,397.8 L227.7,398.9 L228.5,399.7 L229.4,400.4 L230.2,401.4 L231.1,402.4
 L231.9,403.1 L232.8,403.6 L233.6,403.6 L234.5,404.3 L235.3,396.4 L236.2,400.1 L237.0,403.0 L237.9,404.8
 L238.7,405.9 L239.6,406.5 L240.4,406.2 L241.2,403.9 L242.1,403.9 L242.9,404.7 L243.8,405.0 L244.6,405.1
 L245.5,404.9 L246.3,404.2 L247.2,403.6 L248.0,404.0 L248.9,405.0 L249.7,405.9 L250.6,406.7 L251.4,407.2
 L252.3,407.6 L253.1,408.1 L254.0,408.3 L254.8,408.3 L255.7,404.4 L256.5,400.1 L257.4,400.9 L258.2,402.5
 L259.1,403.6 L259.9,404.3 L260.8,404.8 L261.6,405.1 L262.5,405.4 L263.3,405.6 L264.2,405.8 L265.0,406.0
 L265.9,406.2 L266.7,406.4 L267.6,406.5 L268.4,406.7 L269.3,406.9 L270.1,407.1 L271.0,407.2 L271.8,407.4
 L272.7,407.5 L273.5,407.7 L274.4,407.8 L275.2,408.0 L276.1,408.2 L2

## Case 5

In [ ]:
var gp = PlotTableEnrgy[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.01 
 
 
 
 
 0.02 
 
 
 
 
 0.03 
 
 
 
 
 0.04 
 
 
 
 
 0.05 
 
 
 
 
 0.06 
 
 
 
 
 0.07 
 
 
 
 
 0.08 
 
 
 
 
 0.09 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M62.2,512.6 L63.1,512.2 L63.9,512.3 L64.8,512.5 L65.6,512.9 L66.5,513.3
 L67.4,513.7 L68.2,514.2 L69.1,514.7 L69.9,515.2 L70.8,515.8 L71.7,516.3 L72.5,517.8 L73.4,518.6
 L74.2,519.5 L75.1,520.2 L76.0,520.2 L76.8,521.9 L77.7,522.7 L78.5,523.6 L79.4,524.4 L80.3,525.3
 L81.1,526.1 L82.0,527.0 L82.8,527.8 L83.7,528.0 L84.6,528.8 L85.4,529.5 L86.3,530.3 L87.1,531.0
 L88.0,531.7 L88.9,532.4 L89.7,533.1 L90.6,533.8 L91.4,534.4 L92.3,535.0 L93.2,535.7 L94.0,536.2
 L94.9,536.8 L95.7,537.4 L96.6,537.9 L97.5,538.4 L98.3,538.9 L99.2,539.4 L100.0,539.9 L100.9,539.3
 L101.8,539.6 L102.6,539.8 L103.5,542.6 L104.3,543.7 L105.2,544.9 L106.1,546.5 L106.9,531.5 L107.8,543.2
 L108.6,555.6 L109.5,554.3 L110.4,545.8 L111.2,546.1 L112.1,546.4 L112.9,546.7 L113.8,547.0 L114.7,547.3
 L115.5,547.5 L116.4,547.8 L117.2,548.1 L118.1,548.3 L119.0,548.5 L119.8,548.7 L120.7,549.0 L121.5,549.2
 L122.4,549.4 L123.3,549.6 L124.1,549.8 L125.0,549.9 L125.8,550.1 L126.7,550.3 L127.6,550.4 L128.4,550.6
 L129.3,550.8 L130.1,550.9 L131.0,551.1 L131.8,551.2 L132.7,551.3 L133.6,551.5 L134.4,551.6 L135.3,551.7
 L136.1,551.9 L137.0,552.0 L137.9,552.1 L138.7,552.2 L139.6,552.3 L140.4,552.4 L141.3,552.5 L142.2,552.6
 L143.0,552.7 L143.9,552.8 L144.7,552.9 L145.6,553.0 L146.5,553.1 L147.3,553.2 L148.2,553.3 L149.0,553.4
 L149.9,553.5 L150.8,553.6 L151.6,553.7 L152.5,553.7 L153.3,553.8 L154.2,553.9 L155.1,554.0 L155.9,554.1
 L156.8,554.1 L157.6,554.2 L158.5,554.3 L159.4,554.4 L160.2,554.4 L161.1,554.5 L161.9,554.6 L162.8,554.6
 L163.7,554.7 L164.5,554.8 L165.4,554.9 L166.2,554.9 L167.1,555.0 L168.0,555.1 L168.8,555.1 L169.7,555.2
 L170.5,555.2 L171.4,555.3 L172.3,555.4 L173.1,555.4 L174.0,555.5 L174.8,555.6 L175.7,555.6 L176.6,555.7
 L177.4,555.7 L178.3,555.8 L179.1,555.9 L180.0,555.9 L180.9,556.0 L181.7,556.0 L182.6,556.1 L183.4,556.1
 L184.3,556.2 L185.2,556.2 L186.0,556.3 L186.9,556.3 L187.7,556.4 L188.6,556.4 L189.5,556.5 L190.3,556.6
 L191.2,556.6 L192.0,556.6 L192.9,556.7 L193.8,556.7 L194.6,556.8 L195.5,556.8 L196.3,556.9 L197.2,556.9
 L198.1,557.0 L198.9,557.0 L199.8,557.1 L200.6,557.1 L201.5,557.2 L202.4,557.2 L203.2,557.2 L204.1,557.3
 L204.9,557.3 L205.8,557.4 L206.7,557.4 L207.5,557.5 L208.4,557.5 L209.2,557.5 L210.1,557.6 L211.0,557.6
 L211.8,557.7 L212.7,557.7 L213.5,557.7 L214.4,557.8 L215.3,557.8 L216.1,557.8 L217.0,557.9 L217.8,557.9
 L218.7,557.9 L219.6,558.0 L220.4,558.0 L221.3,558.0 L222.1,558.1 L223.0,558.1 L223.9,558.1 L224.7,558.2
 L225.6,558.2 L226.4,558.2 L227.3,558.3 L228.2,558.3 L229.0,558.3 L229.9,558.3 L230.7,558.4 L231.6,558.4
 L232.5,558.4 L233.3,558.5 L234.2,558.5 L235.0,558.5 L235.9,558.5 L236.8,558.6 L237.6,558.6 L238.5,558.6
 L239.3,558.6 L240.2,558.7 L241.1,558.7 L241.9,558.7 L242.8,558.7 L243.6,558.7 L244.5,558.8 L245.4,558.8
 L246.2,558.8 L247.1,558.8 L247.9,558.9 L248.8,558.9 L249.7,558.9 L250.5,558.9 L251.4,558.9 L252.2,558.9
 L253.1,559.0 L254.0,559.0 L254.8,559.0 L255.7,559.0 L256.5,559.0 L257.4,559.1 L258.3,559.1 L259.1,559.1
 L260.0,559.1 L260.8,559.1 L261.7,559.1 L262.6,559.1 L263.4,559.2 L264.3,559.2 L265.1,559.2 L266.0,559.2
 L266.9,559.2 L267.7,559.2 L268.6,559.2 L269.4,559.3 L270.3,559.3 L27

In [ ]:
var gp = PlotTableEnrgy[4,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.15 
 
 
 
 
 3.16 
 
 
 
 
 3.17 
 
 
 
 
 3.18 
 
 
 
 
 3.19 
 
 
 
 
 3.2 
 
 
 
 
 3.21 
 
 
 
 
 3.22 
 
 
 
 
 3.23 
 
 
 
 
 3.24 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M62.2,49.4 L63.1,157.2 L63.9,221.3 L64.8,273.2 L65.6,315.0 L66.5,348.8
 L67.4,376.1 L68.2,398.4 L69.1,416.6 L69.9,431.5 L70.8,443.8 L71.7,453.8 L72.5,463.1 L73.4,470.2
 L74.2,476.4 L75.1,481.5 L76.0,485.1 L76.8,489.7 L77.7,493.0 L78.5,495.9 L79.4,498.5 L80.3,500.9
 L81.1,503.0 L82.0,504.9 L82.8,506.7 L83.7,507.8 L84.6,509.3 L85.4,510.8 L86.3,512.1 L87.1,513.4
 L88.0,514.6 L88.9,515.8 L89.7,516.9 L90.6,518.0 L91.4,519.1 L92.3,520.1 L93.2,521.0 L94.0,522.0
 L94.9,522.9 L95.7,523.7 L96.6,524.6 L97.5,525.4 L98.3,526.1 L99.2,526.9 L100.0,527.6 L100.9,527.4
 L101.8,527.9 L102.6,528.4 L103.5,531.2 L104.3,532.4 L105.2,533.7 L106.1,535.4 L106.9,522.1 L107.8,532.9
 L108.6,544.2 L109.5,543.2 L110.4,535.8 L111.2,536.2 L112.1,536.7 L112.9,537.1 L113.8,537.5 L114.7,537.9
 L115.5,538.3 L116.4,538.7 L117.2,539.0 L118.1,539.4 L119.0,539.7 L119.8,540.0 L120.7,540.3 L121.5,540.6
 L122.4,540.9 L123.3,541.1 L124.1,541.4 L125.0,541.6 L125.8,541.8 L126.7,542.1 L127.6,542.3 L128.4,542.5
 L129.3,542.7 L130.1,542.9 L131.0,543.1 L131.8,543.2 L132.7,543.4 L133.6,543.6 L134.4,543.7 L135.3,543.9
 L136.1,544.0 L137.0,544.2 L137.9,544.3 L138.7,544.4 L139.6,544.5 L140.4,544.7 L141.3,544.8 L142.2,544.9
 L143.0,545.0 L143.9,545.1 L144.7,545.2 L145.6,545.3 L146.5,545.4 L147.3,545.5 L148.2,545.6 L149.0,545.6
 L149.9,545.7 L150.8,545.8 L151.6,545.9 L152.5,546.0 L153.3,546.0 L154.2,546.1 L155.1,546.2 L155.9,546.2
 L156.8,546.3 L157.6,546.4 L158.5,546.4 L159.4,546.5 L160.2,546.5 L161.1,546.6 L161.9,546.7 L162.8,546.7
 L163.7,546.8 L164.5,546.8 L165.4,546.9 L166.2,546.9 L167.1,547.0 L168.0,547.0 L168.8,547.1 L169.7,547.1
 L170.5,547.2 L171.4,547.2 L172.3,547.3 L173.1,547.3 L174.0,547.4 L174.8,547.4 L175.7,547.5 L176.6,547.5
 L177.4,547.6 L178.3,547.6 L179.1,547.7 L180.0,547.7 L180.9,547.8 L181.7,547.8 L182.6,547.8 L183.4,547.9
 L184.3,547.9 L185.2,548.0 L186.0,548.0 L186.9,548.1 L187.7,548.1 L188.6,548.1 L189.5,548.2 L190.3,548.2
 L191.2,548.3 L192.0,548.3 L192.9,548.4 L193.8,548.4 L194.6,548.4 L195.5,548.5 L196.3,548.5 L197.2,548.6
 L198.1,548.6 L198.9,548.6 L199.8,548.7 L200.6,548.7 L201.5,548.7 L202.4,548.8 L203.2,548.8 L204.1,548.9
 L204.9,548.9 L205.8,548.9 L206.7,549.0 L207.5,549.0 L208.4,549.1 L209.2,549.1 L210.1,549.1 L211.0,549.2
 L211.8,549.2 L212.7,549.2 L213.5,549.3 L214.4,549.3 L215.3,549.3 L216.1,549.4 L217.0,549.4 L217.8,549.4
 L218.7,549.5 L219.6,549.5 L220.4,549.5 L221.3,549.6 L222.1,549.6 L223.0,549.6 L223.9,549.7 L224.7,549.7
 L225.6,549.7 L226.4,549.8 L227.3,549.8 L228.2,549.8 L229.0,549.9 L229.9,549.9 L230.7,549.9 L231.6,550.0
 L232.5,550.0 L233.3,550.0 L234.2,550.1 L235.0,550.1 L235.9,550.1 L236.8,550.1 L237.6,550.2 L238.5,550.2
 L239.3,550.2 L240.2,550.2 L241.1,550.3 L241.9,550.3 L242.8,550.3 L243.6,550.4 L244.5,550.4 L245.4,550.4
 L246.2,550.4 L247.1,550.5 L247.9,550.5 L248.8,550.5 L249.7,550.5 L250.5,550.5 L251.4,550.6 L252.2,550.6
 L253.1,550.6 L254.0,550.6 L254.8,550.7 L255.7,550.7 L256.5,550.7 L257.4,550.7 L258.3,550.7 L259.1,550.8
 L260.0,550.8 L260.8,550.8 L261.7,550.8 L262.6,550.8 L263.4,550.9 L264.3,550.9 L265.1,550.9 L266.0,550.9
 L266.9,550.9 L267.7,551.0 L268.6,551.0 L269.4,